<h1>Notes From PostgreSQL Documentation (V17)</h1>

- The main repository for the notebook file is available at [https://github.com/omeraytekinn/postgresql-notes](https://github.com/omeraytekinn/postgresql-notes).
- You can download the notebook file from this repository and run the queries locally.
- For more details you can check [PostgreSQL Documentation](https://www.postgresql.org/docs/17/index.html).
- If you believe there is any incorrect information, please feel free to notify me at [omeraytekin00@gmail.com](mailto:omeraytekin00@gmail.com). I appreciate your feedback!

CONTENTS
- [Installing Dependencies](#Installing-Dependencies)  
- [Initializing Notebook and Connections](#Initializing-Notebook-and-Connections)  
- [Chapter 5. Data Definition](#Chapter-5.-Data-Definition)  
  - [5.1. Table Basics](#5.1.-Table-Basics)  
  - [5.3. Identity Columns](#5.3.-Identity-Columns)  
  - [5.4. Generated Columns](#5.4.-Generated-Columns)  
  - [5.5. Constraints](#5.5.-Constraints)  
    - [5.5.1. Check Constraints](#5.5.1.-Check-Constraints)  
    - [Constraint property DEFERRABLE (Independent from book)](#constraint-property-deferrable-independent-from-book)  
    - [5.5.2. Not-Null Constraints](#5.5.2.-Not-Null-Constraints)  
    - [5.5.3. Unique Constraints](#5.5.3.-Unique-Constraints)  
    - [5.5.4. Primary Keys](#5.5.4.-Primary-Keys)  
    - [5.5.5. Foreign Keys](#5.5.5.-Foreign-Keys)  
    - [5.5.6. Exclusion Constraints](#5.5.6.-Exclusion-Constraints)  
  - [5.7. Modifying Tables](#5.7.-Modifying-Tables)  
    - [5.7.1. Adding a Column](#5.7.1.-Adding-a-Column)  
    - [5.7.2. Removing a Column](#5.7.2.-Removing-a-Column)  
    - [5.7.3. Adding a Constraint](#5.7.3.-Adding-a-Constraint)  
    - [5.7.4. Removing a Constraint](#5.7.4.-Removing-a-Constraint)  
    - [5.7.5. Changing a Column's Default Value](#5.7.5.-Changing-a-Column's-Default-Value)  
    - [5.7.6. Changing a Column's Data Type](#5.7.6.-Changing-a-Column's-Data-Type)  
    - [5.7.7. Renaming a Column](#5.7.7.-Renaming-a-Column)  
    - [5.7.8. Renaming a Table](#5.7.8.-Renaming-a-Table)  
  - [5.8. Privileges](#5.8.-Privileges) 

## Installing Dependencies

<b>Install dependencies and after installment finished restart the notebook.</b>

In [1]:
#!pip install jupyter ipython-sql jupyterlab_sql_editor[all]

## Initializing Notebook and Connections

**This steps must be executed every start/restart of kernel or notebook**

In [1]:
%load_ext sql

In [2]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

Connecting to DB
Usage: &lt;db_vendor&gt;://[username:password]@&lt;serverip&gt;/&lt;dbname&gt;

In [4]:
%sql postgresql://postgres@localhost/mydb

Testing connection

In [5]:
%sql select 'CONNECTION SUCCESSFULLY INITIALIZED' as result

 * postgresql://postgres@localhost/mydb
1 rows affected.


result
CONNECTION SUCCESSFULLY INITIALIZED


## Chapter 5. Data Definition

### 5.1. Table Basics

##### a. Creating Table

In [6]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

##### b. Deleting Table

For details check reference <a href="https://www.postgresql.org/docs/current/sql-droptable.html">https://www.postgresql.org/docs/current/sql-droptable.html</a>

- IF EXISTS is optional. This statement prevents throwing error when table does not exist in DB.

In [7]:
%%sql
DROP TABLE IF EXISTS products CASCADE;

 * postgresql://postgres@localhost/mydb
Done.


[]

##### c. Creating Table with DEFAULT value

In [8]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric DEFAULT 9.99,
	date timestamp DEFAULT CURRENT_TIMESTAMP
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

### 5.3. Identity Columns

##### a. Creating table with IDENTITY column

- When using <i>IDENTITY</i> column with <i>GENERATED</i> it means value of column will be created automatically
- <i>GENERATED ALWAYS</i> means you can not insert manual values when adding/updating for this field.
- <i>GENERATED BY DEFAULT</i> means you can insert manual values when adding/updating for this field
- When using <i>GENERATED BY DEFAULT</i> you must be careful:
    - Generating values will be using a sequential. That means it starts from 1 and goes on.
    - If identity column is described as UNIQUE:
        - You insert a record without inserting a manual value for identity column.
        - After that you give identity column value by hand and that value is 2.
            - It will be successfully inserted.
            - But sequence value is still remains as 2.
        - Then you inserted a record without inserting a manual value for identity column again.
            - Since sequence value is 2 and there is a record also has valued 2, it will throw duplicated value error.
    - If identity column is not described as UNIQUE:
        - You insert a record without inserting a manual value for identity column.
        - After that you give identity column value by hand and that value is 2.
            - It will be successfully inserted.
            - But sequence value is still remains as 2.
        - Then you inserted a record without inserting a manual value for identity column again.
            - Since sequence value is 2 it will insert record with identity column has value as 2.
            - But there are two record which their identity columns has value as 2.
- Even when using <i>GENERATED ALWAYS</i> there are duplicated value or error risks:
    - By using OVERRIDING SYSTEM VALUE, you can enter identity column values manually.
    - Same cases above (GENERATED BY DEFAULT) are also effective for this. 

##### b. Example usage of <i>GENERATED ALWAYS</i>

- Creating table with GENERATED ALWAYS AS IDENTITY column

In [9]:
%%sql
DROP TABLE IF EXISTS people CASCADE;
CREATE TABLE people (
	id bigint GENERATED ALWAYS AS IDENTITY,
	name varchar(80),
	address varchar(80)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Record will be inserted successfully and id will be automatically assigned as 1

In [10]:
%%sql 
INSERT INTO people (name, address) VALUES ('A', 'foo');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


id,name,address
1,A,foo


- Throws error because of id column is generated always and given an id value

In [11]:
%sql INSERT INTO people (id, name, address) VALUES (2, 'B', 'bar');

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.GeneratedAlways) HATA:  cannot insert a non-DEFAULT value into column "id"
DETAIL:  Column "id" is an identity column defined as GENERATED ALWAYS.
HINT:  Geçersiz kılmak (override) için OVERRIDING SYSTEM VALUE kullanın.

[SQL: INSERT INTO people (id, name, address) VALUES (2, 'B' , 'bar' );]
(Background on this error at: https://sqlalche.me/e/20/f405)


- Will not throw error and insert record successfully

In [12]:
%%sql 
INSERT INTO people (id, name, address) OVERRIDING SYSTEM VALUE VALUES (2, 'B', 'bar');

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

- Insert will be successful but since sequence for id value is still 2 and there is a record which id is 2, there is two record which has id 2.

In [13]:
%%sql 
INSERT INTO people (name, address) VALUES ('C', 'baz');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
3 rows affected.


id,name,address
1,A,foo
2,B,bar
2,C,baz


##### c. Example usage of <i>GENERATED BY DEFAULT with UNIQUE descriptor</i>

- Creating table with GENERATED BY DEFAULT AS IDENTITY column

In [14]:
%%sql
DROP TABLE IF EXISTS people CASCADE;
CREATE TABLE people (
	id bigint UNIQUE GENERATED BY DEFAULT AS IDENTITY,
	name varchar(80),
	address varchar(80)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- First record will be inserted successfully and id will be automatically assigned as 1.
- Second record will be inserted successfully.

In [15]:
%%sql 
INSERT INTO people (name, address) VALUES ('A', 'foo');
INSERT INTO people (id, name, address) VALUES (2, 'B', 'bar');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
2 rows affected.


id,name,address
1,A,foo
2,B,bar


- Error will be thrown because sequence for id value is still 2 and there is a record which id is 2

In [16]:
%%sql 
INSERT INTO people (name, address) VALUES ('C', 'baz');

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "people_id_key" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(id)=(2)" anahtarı zaten mevcut

[SQL: INSERT INTO people (name, address) VALUES ('C', 'baz');]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### d. Example usage of <i>GENERATED BY DEFAULT without UNIQUE descriptor</i>

- Creating table with GENERATED BY DEFAULT AS IDENTITY column

In [17]:
%%sql
DROP TABLE IF EXISTS people CASCADE;
CREATE TABLE people (
	id bigint GENERATED BY DEFAULT AS IDENTITY,
	name varchar(80),
	address varchar(80)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- First record will be inserted successfully and id will be automatically assigned as 1.
- Second record will be inserted successfully.

In [18]:
%%sql 
INSERT INTO people (name, address) VALUES ('A', 'foo');
INSERT INTO people (id, name, address) VALUES (2, 'B', 'bar');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
2 rows affected.


id,name,address
1,A,foo
2,B,bar


- Insert will be successful but since sequence for id value is still 2 and there is a record which id is 2, there is two record which has id 2.

In [19]:
%%sql 
INSERT INTO people (name, address) VALUES ('C', 'baz');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
3 rows affected.


id,name,address
1,A,foo
2,B,bar
2,C,baz


##### e. Usage of DEFAULT value

- DEFAULT also can be used to specify the sequence explicitly.
- Especially in UPDATE command when update generated column value and get it from sequence, you can use DEFAULT.

- Creating table has generated column

In [20]:
%%sql
DROP TABLE IF EXISTS people CASCADE;
CREATE TABLE people (
	id bigint GENERATED ALWAYS AS IDENTITY,
	name varchar(80),
	address varchar(80)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting values.
- When inserting values id will be fetched from a sequence.
- First record has id 1, and second has id 2.
- After insertions completed, current value of sequence will be 3.

In [21]:
%%sql
INSERT INTO people (name, address) VALUES ('A', 'foo');
INSERT INTO people (name, address) VALUES ('B', 'bar');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
2 rows affected.


id,name,address
1,A,foo
2,B,bar


- Inserting value using DEFAULT.
- Using DEFAULT means use sequence to insert value.
- So the value will be current value of sequence (3) and current value of sequence increases.

In [22]:
%%sql 
INSERT INTO people (id, name, address) VALUES (DEFAULT, 'C', 'baz');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
3 rows affected.


id,name,address
1,A,foo
2,B,bar
3,C,baz


- DEFAULT value also can be used in UPDATE statement.
- When using update statement, id value will be fetched from sequence still.
- So new id of third record will be 4.
- The other use of DEFAULT is in name column. Default value of name is NULL because it is not specified a default value.

In [23]:
%%sql
UPDATE people SET id=DEFAULT, name=DEFAULT, address='qwe' where id = 3;
SELECT * FROM people; 

 * postgresql://postgres@localhost/mydb
1 rows affected.
3 rows affected.


id,name,address
1,A,foo
2,B,bar
4,None,qwe


### 5.4. Generated Columns

- Syntax is GENERATED [ALWAYS|BY DEFAULT] AS ... STORED
- Creates a calculated column by using a column as base column.
- Example usages:
    - When you have to calculate a value using a column frequently, then you can store the calculated value in another column.
        - By that way you can also index that column and make optimizations.
    - When calculating, if base column is NULL then generated column will be also NULL.
    - (From documentation) Generated columns maintain access privileges separately from their underlying base columns.
        - So, it is possible to arrange it so that a particular role can read from a generated column but not from the underlying base columns.
        - For example you can hide email value from users except current user is admin (for simplicity current_user used instead of role based query).
            - To achive this there are 2 columns for email: email and visible_email
            - visible_email will be generated using email.
            - When getting visible_email if current user is admin email will be seen, but is not admin ***** will be seen in results.
            - To prevent other users get email by using email column, user rights will be revoked.
            - ```sql
                CREATE TABLE users (
                    id SERIAL PRIMARY KEY,
                    email VARCHAR(200) NOT NULL,
                    visible_email VARCHAR(200) GENERATED ALWAYS AS (
                        CASE 
                            WHEN current_user = 'admin' THEN email
                            ELSE '*****'
                        END
                    ) STORED
                );
                REVOKE SELECT ON users FROM PUBLIC;
                GRANT SELECT (id, visible_email) ON users TO PUBLIC;
                GRANT SELECT (id, email, visible_email) ON users TO admin;
              ```
     

##### a. Creating table with GENERATED column

In [24]:
%%sql
DROP TABLE IF EXISTS people CASCADE;
CREATE TABLE people (
	id bigint PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
	name varchar(80),
	address varchar(80),
	height_cm numeric,
	height_in numeric GENERATED ALWAYS AS (height_cm / 2.54) STORED
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting values to table with GENERATED column.
- See that when height_cm is null, generated column is also null.

In [25]:
%%sql
INSERT INTO people (name, address, height_cm) VALUES ('A', 'foo', 170); 
INSERT INTO people (name, address) VALUES ('B', 'bar');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
2 rows affected.


id,name,address,height_cm,height_in
1,A,foo,170,66.9291338582677165
2,B,bar,None,None


### 5.5. Constraints

#### 5.5.1. Check Constraints

##### a. Creating table with check constraint

In [26]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric CHECK (price > 0)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting record to table has check constraint
- Operation will throw error because of price should be greater than zero.

In [27]:
%sql INSERT INTO products VALUES (1, 'product-1', 0);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.CheckViolation) HATA:  "products" tablosuna girilen yeni satır "products_price_check" check kısıtlamasını ihlal ediyor
DETAIL:  Hata veren satır (1, product-1, 0) içeriyor.

[SQL: INSERT INTO products VALUES (1, 'product-1' , 0);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### b. Creating table with check constraint which has name

- Giving a constraint name clarifies error messages and allows you to refer it when constraint needed to change

In [28]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric CONSTRAINT positive_price CHECK (price > 0)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting record to table has named check constraint
- Operation will throw error because of price should be greater than zero.
- See that error message includes constraint name (positive_price).

In [29]:
%sql INSERT INTO products VALUES (1, 'product-1', 0);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.CheckViolation) HATA:  "products" tablosuna girilen yeni satır "positive_price" check kısıtlamasını ihlal ediyor
DETAIL:  Hata veren satır (1, product-1, 0) içeriyor.

[SQL: INSERT INTO products VALUES (1, 'product-1' , 0);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### c. Removing constraint from table and adding new constraint

- First creating table with constraint and give a name to constraint.

In [30]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric CONSTRAINT positive_price CHECK (price > 0)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Removing a constraint and adding new constraint.
- Since constraint has a name it easy to remove using name.
- When not given a name you should find constraint int pg tables. (For more: Google -> PostgreSQL drop constraint with unknown name)

In [31]:
%%sql
ALTER TABLE products DROP CONSTRAINT IF EXISTS positive_price;
ALTER TABLE products ADD CONSTRAINT minimum_price CHECK (price >= 10.0);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

##### d. Other constraint informations

- Check constraint can be added for multiple columns or full table.
- When adding new constraint to non-empty table, all records should complain new constraint. Otherwise constraint is violated by some row error will be thrown. (This is not always the case; details will be provided later.)

- First recreating table for cleaning constraints.

In [32]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric CONSTRAINT positive_price CHECK (price > 0)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Adding constraint using multiple columns.

In [33]:
%sql ALTER TABLE products ADD CONSTRAINT product_no_and_price CHECK (product_no > 0 and price > 0);

 * postgresql://postgres@localhost/mydb
Done.


[]

- Adding constraint for all columns.

In [34]:
%sql ALTER TABLE products ADD CONSTRAINT lock_table CHECK (false);

 * postgresql://postgres@localhost/mydb
Done.


[]

- Check constraing can be applied only new added or modified records.
- To achieve this NOT VALID should be used.

- First cleaning constraints.

In [35]:
%%sql 
ALTER TABLE products DROP CONSTRAINT IF EXISTS product_no_and_price;
ALTER TABLE products DROP CONSTRAINT IF EXISTS lock_table;

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting values to product table to see effect of new constraints on old values.

In [36]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (2, 'product-2', 20);
SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
2 rows affected.


product_no,name,price
1,product-1,10
2,product-2,20


- First see when adding new constraing without NOT VALID property.
- It throws error because there are rows which not comply with new constraint.

In [37]:
%sql ALTER TABLE products ADD CONSTRAINT lock_table CHECK (false);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.CheckViolation) HATA:  check constraint "lock_table" of relation "products" is violated by some row

[SQL: ALTER TABLE products ADD CONSTRAINT lock_table CHECK (false);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- Now when using NOT VALID, constraint does not check old values
- So there will be no error thrown.

In [38]:
%sql ALTER TABLE products ADD CONSTRAINT lock_table CHECK (false) NOT VALID;

 * postgresql://postgres@localhost/mydb
Done.


[]

- You can check the constraint for old values manually.
- To do that you can use VALIDATE command.
- This statement throws error if there is any record that violates new constraint.

In [39]:
%sql ALTER TABLE products VALIDATE CONSTRAINT lock_table;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.CheckViolation) HATA:  check constraint "lock_table" of relation "products" is violated by some row

[SQL: ALTER TABLE products VALIDATE CONSTRAINT lock_table;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- Also when inserting new value to table it will throw error.
- Because constraint does not allow new records.

In [40]:
%sql INSERT INTO products VALUES (3, 'product-3', 30);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.CheckViolation) HATA:  "products" tablosuna girilen yeni satır "lock_table" check kısıtlamasını ihlal ediyor
DETAIL:  Hata veren satır (3, product-3, 30) içeriyor.

[SQL: INSERT INTO products VALUES (3, 'product-3' , 30);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


#### Constraint property DEFERRABLE (Independent from book)

- For constraints there is also DEFERRABLE property is available.
- DEFERRABLE specifies action timing for constraints.
- It specifies will constraint applied immediately or applied end of transaction.
- Usage is: CONSTRAINT ... [ DEFERRABLE | NOT DEFERRABLE ] [ INITIALLY DEFERRED | INITIALLY IMMEDIATE ]
    - [ DEFERRABLE | NOT DEFERRABLE ]: default value is NOT DEFERRABLE.
    - [ INITIALLY DEFERRED | INITIALLY IMMEDIATE ]: default value is INITIALLY IMMEDIATE.
- INITIALLY DEFERRED means in all operations constraint will be checked immediately before insert/update.
- INITIALLY DEFERRED means checking constraint wil be deferred until end of transaction (commit, rollback or error states).
- Actually this cases will be initial case for DEFERRABLE. That means you can change DEFERRED or IMMEDIATE status in the transaction. But this change is only valid in transaction. Does not affect other transactions.
    - For example when defining foreign key constraint with DEFERRABLE without specifying immediate or deferred, its default value is INITIALLY IMMEDIATE.
        - SET CONSTRAINT ... DEFERRED
    - So when in transaction if foreign key will be given later in the transaction, you can set status to DEFERRED and foreign key constraint will be checked end of transaction.
        - SET CONSTRAINT ... IMMEDIATE
    - The reverse could have also been done. A constraint can be defined INITIALLY DEFERRED and you may want to see result of an operation without waiting end of transaction. So you can change it to IMMEDIATE in the transaction. Then constraint will be checked immediately.
- If NOT DEFERRABLE used, you can not change the defer status in the transaction temporarily.
- Yes it can be changed by using ALTER TABLE statement but if affects all latter transactions. Since this action involves modifying the database schema, it is not recommended to use it indiscriminately.
    - ALTER TABLE ... DROP CONSTRAINT ...;
    - ALTER TABLE ... ADD CONSTRAINT ... FOREIGN KEY (...) REFERENCES ... DEFERRABLE INITIALLY DEFERRED;

###### e.1 DEFERRABLE NOT DEFERRABLE usage

- To see the effects in this section auto-commit will be disabled.
- Do not forget to re-open for other sections.

In [41]:
%config SqlMagic.autocommit=False

- Since default value for deferration is NOT DEFERRABLE, unique_product_no constraint will be NOT DEFERRABLE 

In [42]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
	product_no integer CONSTRAINT unique_product_no UNIQUE,
	name text,
	price numeric
);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Second insertion will throw error immediately without waiting the transaction ends. 
- Since error thrown in transaction, it rolls back and none of the insertions will be reflected in the table.

In [43]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (2, 'product-2', 20);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
1 rows affected.
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "unique_product_no" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(product_no)=(1)" anahtarı zaten mevcut

[SQL: INSERT INTO products VALUES (1, 'product-1', 10);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [44]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
0 rows affected.


product_no,name,price


In [45]:
%config SqlMagic.autocommit=True

###### e.2 DEFERRABLE INITIALLY DEFERRED usage

- To see the effects in this section auto-commit will be disabled.
- Do not forget to re-open for other sections.

In [46]:
%config SqlMagic.autocommit=False

- Creating table with INITIALLY DEFERRED statement.

In [47]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
	product_no integer CONSTRAINT unique_product_no UNIQUE DEFERRABLE INITIALLY DEFERRED,
	name text,
	price numeric
);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- In this transaction second insertion will not throw error. Because unique constraint check deferred until transaction ends.
- In third statement products with id 1 are deleted.
- And in fourth statement a product with id 1 is inserted to empty table.
- At the end of transaction there is only 1 record which has id 1.
- So transaction commit successfully because all records comply with unique constraint.

In [48]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (1, 'product-1', 10);
DELETE FROM products where product_no=1;
INSERT INTO products VALUES (1, 'product-1', 10);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
1 rows affected.
1 rows affected.
2 rows affected.
1 rows affected.
Done.


[]

- In the end there is only 1 record in table.

In [49]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
1 rows affected.


product_no,name,price
1,product-1,10


- An example that gives an error.
- But the error will be thrown at the end of transaction.

In [50]:
%%sql
BEGIN;
INSERT INTO products VALUES (2, 'product-2', 10);
INSERT INTO products VALUES (2, 'product-2', 10);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
1 rows affected.
1 rows affected.
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "unique_product_no" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(product_no)=(2)" anahtarı zaten mevcut

[SQL: COMMIT;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [51]:
%config SqlMagic.autocommit=True

###### e.3 Changing DEFERRABLE status in transaction

- To see the effects in this section auto-commit will be disabled.
- Do not forget to re-open for other sections.

In [52]:
%config SqlMagic.autocommit=False

- First creating a table with DEFERRABLE INITIALLY IMMEDIATE

In [53]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
	product_no integer CONSTRAINT unique_product_no UNIQUE DEFERRABLE INITIALLY IMMEDIATE,
	name text,
	price numeric
);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- In this transaction after adding a record to table changed CONSTRAINT status to DEFERRED.
- When trying to add new record with same id do not throw error, because constraint deferred until transaction ends.
- Since no error thrown, deleting records with id 1, adding new records with id 2 and 1 execute successfully.
- When transaction ends unique constraint check validates that all records.
- Since all records comply with constraint there is 2 record in the table in the end.

In [54]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
SET CONSTRAINTS unique_product_no DEFERRED;
INSERT INTO products VALUES (1, 'product-1', 10);
DELETE FROM products WHERE product_no = 1;
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (2, 'product-2', 20);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
1 rows affected.
Done.
1 rows affected.
2 rows affected.
1 rows affected.
1 rows affected.
Done.


[]

In [55]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
2 rows affected.


product_no,name,price
1,product-1,10
2,product-2,20


In [56]:
%config SqlMagic.autocommit=True

###### e.4 Changing DEFERRABILITY

- To see the effects in this section auto-commit will be disabled.
- Do not forget to re-open for other sections.

In [57]:
%config SqlMagic.autocommit=False

- Not specified deferrability so it is NOT DEFERRABLE

In [58]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
	product_no integer CONSTRAINT unique_product_no UNIQUE,
	name text,
	price numeric
);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Since unique_product_no is not deferrable, when change constraint to DEFERRED will cause error.

In [59]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
SET CONSTRAINTS unique_product_no DEFERRED;
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
1 rows affected.
(psycopg2.errors.WrongObjectType) HATA:  "unique_product_no" constrainti ertelenebilir constraint değildir

[SQL: SET CONSTRAINTS unique_product_no DEFERRED;]
(Background on this error at: https://sqlalche.me/e/20/f405)


- To make NOT DEFERRABLE constraint to DEFERRED, ALTER TABLE used.
- This is not temporarily in transaction and affects all latter transactions.

In [60]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
ALTER TABLE products DROP CONSTRAINT IF EXISTS unique_product_no;
ALTER TABLE products ADD CONSTRAINT unique_product_no UNIQUE(product_no) DEFERRABLE INITIALLY DEFERRED;
INSERT INTO products VALUES (1, 'product-1', 10);
DELETE FROM products WHERE product_no = 1;
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (2, 'product-2', 20);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
1 rows affected.
Done.
Done.
1 rows affected.
2 rows affected.
1 rows affected.
1 rows affected.
Done.


[]

In [61]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
2 rows affected.


product_no,name,price
1,product-1,10
2,product-2,20


In [62]:
%config SqlMagic.autocommit=True

#### 5.5.2. Not-Null Constraints

- <i>NOT NULL</i> constraint is a column constraint. You can not write it as table constraint.
    - <i>product_no integer NOT NULL</i> is correct usage.
    - But there is no usage like this: <del><i>CONSTRAINT not_null NOT NULL (product_no)</i></del>.
- <i>product_no integer NOT NULL</i> and <i>CONSTRAINT product_no_not_null CHECK (product_no IS NOT NULL)</i> is functionally equivalent. But NOT NULL constraint is more optimized.
    - With using CONSTRAINT it became table constraing and while defining explicit name it can be removed bey using name.
        - <i>ALTER TABLE ... DROP CONSTRAINT ...;</i>
- Since <i>NOT NULL</i> is column constraint and it can not be explicitly named, you can remove it with using <i>ALTER TABLE ... ALTER COLUMN ... DROP NUT NULL;</i>
- In column there can be multiple constraints.
    - Example: <i>price numeric NOT NULL CHECK (price > 0)</i>

- NOT NULL example

In [63]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
    product_no integer NOT NULL,
    name text NOT NULL,
    price numeric
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Violating NOT NULL constraint

In [64]:
%sql INSERT INTO products(price) VALUES (10);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.NotNullViolation) HATA:  null value in column "product_no" of relation "products" violates not-null constraint
DETAIL:  Hata veren satır (null, null, 10) içeriyor.

[SQL: INSERT INTO products(price) VALUES (10);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- Remove not null constraint example

In [65]:
%%sql
ALTER TABLE products 
    ALTER COLUMN product_no DROP NOT NULL,
    ALTER COLUMN name DROP NOT NULL;

 * postgresql://postgres@localhost/mydb
Done.


[]

- Not violating after dropping NOT NULL constraint

In [66]:
%sql INSERT INTO products(price) VALUES (10);

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

- Multiple constraints in column example

In [67]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
    product_no integer NOT NULL,
    name text NOT NULL,
    price numeric NOT NULL CHECK (price > 0)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

#### 5.5.3. Unique Constraints

- <i>UNIQUE</i> constraint can be written either as a column constraint or a table constraint.
    - Table constraint: <i>product_no integer CONSTRAINT unique_product_no UNIQUE</i>
        - Table constraint can be removed by using <i>ALTER TABLE ... DROP CONSTRAINT ...;</i>
    - Column constraint: <i>product_no integer CONSTRAINT unique_product_no UNIQUE</i>
        - Column constraint can not be removed by using <del><i>ALTER TABLE ... ALTER COLUMN DROP UNIQUE;</i></del>
        - To remove constraint you first find the constraint name and remove with <i>ALTER TABLE ... DROP CONSTRAINT ...;</i>
- <i>UNIQUE</i> constraint can be written for one column or multiple column group.
    - For one column: <i>UNIQUE (product_no)</i>. This ensures product_no is unique.
    - For multiple column: <i>UNIQUE (a,c)</i>. This ensures a and c pair is unique.
        - For example when (a,c) = (1,1) (a,c) = (1,2) (a,c) = (2,1) can be inserted but (a,c) = (1,1) can not be inserted again.
- When using UNIQUE constraint, a UNIQUE B-tree index will be added to table.
- If you want to add UNIQUE constraint for only filtered rows you hould use partial index:
  - <i>CREATE UNIQUE INDEX unique_active_product_no ON products (product_no) WHERE active = true;</i>
- <i>UNIQUE</i> constraint on NULL values:
    - NULL values will be seen as different in UNIQUE columns. So you can insert multiple NULL values for a UNIQUE column and you get no violation of constraint error.
    - To prevent this situation you can use <i>NULLS NOT DISTINCT</i> statement: "<i>product_no integer UNIQUE NULLS NOT DISTINCT</i>".

##### Creating UNIQUE constraint for single column

In [68]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
    product_no integer UNIQUE,
    name text,
    price numeric
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Violating unique constraint

In [69]:
%%sql
INSERT INTO products VALUES (1, 'name-1', 10);
INSERT INTO products VALUES (1, 'name-1', 10);

 * postgresql://postgres@localhost/mydb
1 rows affected.
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "products_product_no_key" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(product_no)=(1)" anahtarı zaten mevcut

[SQL: INSERT INTO products VALUES (1, 'name-1', 10);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### Creating UNIQUE constraint for column pair

In [70]:
%%sql
DROP TABLE IF EXISTS example CASCADE;
CREATE TABLE example (
    a integer,
    b integer,
    c integer,
    UNIQUE (a, c)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting values

In [71]:
%%sql
INSERT INTO example (a,c) VALUES ('1', '1');
INSERT INTO example (a,c) VALUES ('1', '2');
INSERT INTO example (a,c) VALUES ('2', '1');

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
1 rows affected.


[]

- Violating constraint

In [72]:
%sql INSERT INTO example (a,c) VALUES ('1', '1');

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "example_a_c_key" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(a, c)=(1, 1)" anahtarı zaten mevcut

[SQL: INSERT INTO example (a,c) VALUES ('1', '1' );]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### Removing UNIQUE constraint

In [73]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
    product_no integer,
    name text,
    price numeric,
    CONSTRAINT unique_product_no UNIQUE (product_no)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

In [74]:
%sql ALTER TABLE products DROP CONSTRAINT IF EXISTS unique_product_no;

 * postgresql://postgres@localhost/mydb
Done.


[]

##### Creating UNIQUE constraint for filtered records using partial INDEX

In [75]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
    product_no integer,
    name text,
    price numeric,
    active boolean
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

In [76]:
%sql CREATE UNIQUE INDEX unique_index_products_product_no_active ON products (product_no) WHERE active = true;

 * postgresql://postgres@localhost/mydb
Done.


[]

- Violating constraint

In [77]:
%%sql
INSERT INTO products (product_no, active) VALUES (1, true);
INSERT INTO products (product_no, active) VALUES (2, false);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

- This throws error because there is active product 1 

In [78]:
%sql INSERT INTO products (product_no, active) VALUES (1, true);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "unique_index_products_product_no_active" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(product_no)=(1)" anahtarı zaten mevcut

[SQL: INSERT INTO products (product_no, active) VALUES (1, true);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- This is successful because there is product 2 but it is not active

In [79]:
%sql INSERT INTO products (product_no, active) VALUES (2, true);

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

##### Using NULL values without DISTINCT restriction

In [80]:
%%sql
DROP TABLE IF EXISTS users CASCADE;
CREATE TABLE users (
    email TEXT UNIQUE
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting multiple NULL values does not throw error

In [81]:
%%sql
INSERT INTO users VALUES (NULL);
INSERT INTO users VALUES (NULL);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

In [82]:
%sql SELECT * FROM users;

 * postgresql://postgres@localhost/mydb
2 rows affected.


email
None
None


##### Using NULL values with DISTINCT restriction

In [83]:
%%sql
DROP TABLE IF EXISTS users CASCADE;
CREATE TABLE users (
    email TEXT UNIQUE NULLS NOT DISTINCT
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Violating UNIQUE constraint because NULLs are not distinct

In [84]:
%%sql
INSERT INTO users VALUES (NULL);
INSERT INTO users VALUES (NULL);

 * postgresql://postgres@localhost/mydb
1 rows affected.
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "users_email_key" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(email)=(null)" anahtarı zaten mevcut

[SQL: INSERT INTO users VALUES (NULL);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


#### 5.5.4. Primary Keys

- <i>PRIMARY KEY</i> can be defined for single column or column pairs.
    - <i>product_no integer PRIMARY KEY,</i>
    - <i>PRIMARY KEY(a,c)</i>
- Tables can have at most one primary key.
- Behaviour of <i>PRIMARY KEY</i> and <i>UNIQUE NOT NULL</i> is equivalent.
- Primary key creates B-tree index for interested column (or pair of columns).

##### Single column PRIMARY KEY

In [85]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- See index for PK

In [86]:
%sql SELECT indexname, indexdef FROM pg_indexes WHERE tablename = 'products';

 * postgresql://postgres@localhost/mydb
1 rows affected.


indexname,indexdef
products_pkey1,CREATE UNIQUE INDEX products_pkey1 ON public.products USING btree (product_no)


- Violation of PK constraint

In [87]:
%%sql
INSERT INTO products (product_no) VALUES (1);
INSERT INTO products (product_no) VALUES (1);

 * postgresql://postgres@localhost/mydb
1 rows affected.
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "products_pkey1" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(product_no)=(1)" anahtarı zaten mevcut

[SQL: INSERT INTO products (product_no) VALUES (1);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### Pair of columns PRIMARY KEY

In [88]:
%%sql
DROP TABLE IF EXISTS example CASCADE;
CREATE TABLE example (
    a integer,
    b integer,
    c integer,
    PRIMARY KEY (a,b)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting values

In [89]:
%%sql
INSERT INTO example (a,b) VALUES (1,1);
INSERT INTO example (a,b) VALUES (1,2);
INSERT INTO example (a,b) VALUES (2,1);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
1 rows affected.


[]

- Violation of PK

In [90]:
%sql INSERT INTO example (a,b) VALUES (1,1);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "example_pkey" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(a, b)=(1, 1)" anahtarı zaten mevcut

[SQL: INSERT INTO example (a,b) VALUES (1,1);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### Adding PK with ALTER TABLE

In [91]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
    product_no integer,
    name text,
    price numeric
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

In [92]:
%sql ALTER TABLE products ADD CONSTRAINT pk_products_product_no PRIMARY KEY (product_no);

 * postgresql://postgres@localhost/mydb
Done.


[]

#### 5.5.5. Foreign Keys

- <i>FOREIGN KEY</i> constraint specifies <i>referential integrity</i> between related tables.
- FK can be used with specifying column name or only specifying table name.
    - <i>product_no integer REFERENCES products (product_no)</i>
    - <i>product_no integer REFERENCES products</i>
- FOREIGN KEY does not create index automatically for interested column. To optimize querying on foreign key columns you can create an index.
- FK can be specified for non-primary columns. But referenced column must have UNIQUE constraint.
- You can reference same table in FK.
- A table can have multiple foreign key constraints.
    - This case can be used for many-to-many relationships.
- When deleting a record which have references there are 5 actions:
    - NO ACTION (default action):
        - Default action if there is is not any action specified is NO ACTION.
        - But you can define this action explicitly.
        - This action prevents deleting any record which has references on other tables.
    - RESTRICT (default action):
        - This action is like NO ACTION which prevents deleting any record which has references on other tables.
        - The difference between NO ACTION and RESTRICT is in transaction:
            - NO ACTION can be defined as deferrable.
                - <i>product_no integer REFERENCES products(product_no) ON DELETE NO ACTION DEFERRABLE INITIALLY DEFERRED,</i>
                - Waits until end of transaction to apply FK constraint.
            - RESTRICT can not be defined as deferrable.
                - PostgreSQL allows you to define RESTRICT with DEFERRABLE, but it is not effective. It still behaves as restricted and take action immediately.
    - CASCADE:
        - This action deletes record with reference records.
    - SET NULL:
        - This action sets value of referenced column as NULL.
    - SET DEFAULT:
        - This action sets value of referenced column as DEFAULT value.
        - Using SET DEFAULT is risky.
        - When setting default value of fk column, default value must reference any product otherwise error will be raised.

##### Usage of FOREIGN KEY constraint

- Defining FK constraint

In [93]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer REFERENCES products (product_no),
    quantity integer
)

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting product and related order 

In [94]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO orders VALUES (1, 1, 10)

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

- Inserting order which is not related with any product

In [95]:
%sql INSERT INTO orders VALUES (2, NULL, 10)

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

- Inserting order with a product which not exists will throw error

In [96]:
%sql INSERT INTO orders VALUES (3, 2, 10)

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.ForeignKeyViolation) HATA:  "orders" tablosu üzerindeki ekleme veya güncelleme işlemi "orders_product_no_fkey" foreign key kısıtlamasını ihlal ediyor
DETAIL:  "products" tablosunda (product_no)=(2) anahtarı mevcut değildir.

[SQL: INSERT INTO orders VALUES (3, 2, 10)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### Usage of FOREIGN KEY constraint as table constraint

In [97]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer ,
    quantity integer,
    FOREIGN KEY (product_no) REFERENCES products(product_no)
)

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

##### Usage of FOREIGN KEY constraint for non-primary columns

- For non-unique referenced columns, error will be thrown

In [98]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer ,
    quantity integer,
    FOREIGN KEY (product_no) REFERENCES products(product_no)
)

 * postgresql://postgres@localhost/mydb
Done.
Done.
(psycopg2.errors.InvalidForeignKey) HATA:  "products" referans edilen tablosunda belirtilen anahtarlara uyan bir unique constraint yok

[SQL: CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer ,
    quantity integer,
    FOREIGN KEY (product_no) REFERENCES products(product_no)
)]
(Background on this error at: https://sqlalche.me/e/20/f405)


- For unique referenced columns, adding fk constraint will be successful

In [99]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer UNIQUE,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer ,
    quantity integer,
    FOREIGN KEY (product_no) REFERENCES products(product_no)
)

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- In contrast to a primary key column reference, when using a non-primary column for reference, you should specify the column name.
- In that case, an error will be thrown.

In [100]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer UNIQUE,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer ,
    quantity integer,
    FOREIGN KEY (product_no) REFERENCES products
)

 * postgresql://postgres@localhost/mydb
Done.
Done.
(psycopg2.errors.UndefinedObject) HATA:  referans edilen "products" tablosunda primary key mevcut değil

[SQL: CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer ,
    quantity integer,
    FOREIGN KEY (product_no) REFERENCES products
)]
(Background on this error at: https://sqlalche.me/e/20/f405)


##### Usage of FOREIGN KEY for pair of columns

In [101]:
%%sql
DROP TABLE IF EXISTS t1, t2 CASCADE;
CREATE TABLE t1 (
    a integer,
    b integer,
    PRIMARY KEY (a,b)
);
CREATE TABLE t2 (
    q integer PRIMARY KEY,
    w integer,
    e integer,
    FOREIGN KEY (w, e) REFERENCES t1 (a, b)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting records for referenced tables

In [102]:
%%sql
INSERT INTO t1 VALUES (1,1);
INSERT INTO t1 VALUES (1,2);
INSERT INTO t1 VALUES (2,1);
INSERT INTO t2 VALUES (1,1,1);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

- Inserting record that there is no related record for referenced table will throw error

In [103]:
%sql INSERT INTO t2 VALUES (2,1,3);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.ForeignKeyViolation) HATA:  "t2" tablosu üzerindeki ekleme veya güncelleme işlemi "t2_w_e_fkey" foreign key kısıtlamasını ihlal ediyor
DETAIL:  "t1" tablosunda (w, e)=(1, 3) anahtarı mevcut değildir.

[SQL: INSERT INTO t2 VALUES (2,1,3);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### Self referencing in FOREIGN KEY

- A table column with a foreign key can reference the same table.

In [104]:
%%sql
DROP TABLE IF EXISTS tree CASCADE;
CREATE TABLE tree (
    node_id integer PRIMARY KEY,
    parent_id integer REFERENCES tree,
    name text
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- In this example parent of root node will be null

In [105]:
%%sql
INSERT INTO tree VALUES (1, NULL, 'root');
INSERT INTO tree VALUES (2, 1, 'child-1 of 1');
INSERT INTO tree VALUES (3, 1, 'child-2 of 2');

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [106]:
%sql SELECT * FROM tree;

 * postgresql://postgres@localhost/mydb
3 rows affected.


node_id,parent_id,name
1,None,root
2,1,child-1 of 1
3,1,child-2 of 2


##### Multiple FOREIGN KEY constraints

- When to establish many-to-many relationship for order and products, a middle table with referencing this tables can be used.

In [107]:
%%sql
DROP TABLE IF EXISTS products, orders, order_items CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    shipping_address text
);
CREATE TABLE order_items (
    product_no integer REFERENCES products,
    order_id integer REFERENCES orders,
    quantity integer,
    PRIMARY KEY (product_no, order_id)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.
Done.


[]

##### Actions when deleting referenced records

###### Preventing delete action with NO ACTION (Default action)

- When not specifying delete action, it is automatically assigned as NO ACTION
- But you can specify explicitly NO ACTION

In [108]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer REFERENCES products(product_no) ON DELETE NO ACTION,
    quantity integer
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting referenced records

In [109]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO orders VALUES (1, 1, 10);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

- When trying to delete a product which referenced in orders will raise error

In [110]:
%sql DELETE FROM products WHERE product_no = 1;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.ForeignKeyViolation) HATA:  "products" tablosu üzerinde yapılan update veya delete işlemi "orders" tablosunun "orders_product_no_fkey" bütünlük kısıtlamasını ihlal ediyor
DETAIL:  (product_no)=(1) anahtarı "orders" tablosundan hala referans edilmektedir.

[SQL: DELETE FROM products WHERE product_no = 1;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- Deleting related orders before deleting product will be successful

In [111]:
%%sql 
DELETE FROM orders WHERE order_id = 1;
DELETE FROM products WHERE product_no = 1;

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

###### Preventing delete action with RESTRICT

In [112]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer REFERENCES products(product_no) ON DELETE RESTRICT,
    quantity integer
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting referenced records

In [113]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO orders VALUES (1, 1, 10);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

- When trying to delete a product which referenced in orders will raise error

In [114]:
%sql DELETE FROM products WHERE product_no = 1;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.ForeignKeyViolation) HATA:  "products" tablosu üzerinde yapılan update veya delete işlemi "orders" tablosunun "orders_product_no_fkey" bütünlük kısıtlamasını ihlal ediyor
DETAIL:  (product_no)=(1) anahtarı "orders" tablosundan hala referans edilmektedir.

[SQL: DELETE FROM products WHERE product_no = 1;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- Deleting related orders before deleting product will be successful

In [115]:
%%sql 
DELETE FROM orders WHERE order_id = 1;
DELETE FROM products WHERE product_no = 1;

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

###### Deleting related records automatically with CASCADE

In [116]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer REFERENCES products(product_no) ON DELETE CASCADE,
    quantity integer
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting related records

In [117]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO orders VALUES (1, 1, 10);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

In [118]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
1 rows affected.


product_no,name,price
1,product-1,10


In [119]:
%sql SELECT * FROM orders;

 * postgresql://postgres@localhost/mydb
1 rows affected.


order_id,product_no,quantity
1,1,10


- Deleting product which has related orders will cause deleting orders

In [120]:
%sql DELETE FROM products WHERE product_no = 1;

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

- See that both product and order is deleted

In [121]:
%sql SELECT * from products;

 * postgresql://postgres@localhost/mydb
0 rows affected.


product_no,name,price


In [122]:
%sql SELECT * FROM orders;

 * postgresql://postgres@localhost/mydb
0 rows affected.


order_id,product_no,quantity


###### Deleting related records automatically with set NULL

In [123]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer REFERENCES products(product_no) ON DELETE SET NULL,
    quantity integer
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting related records

In [124]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO orders VALUES (1, 1, 10);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

- Deleting product will cause setting product_no of related order to NULL

In [125]:
%sql DELETE FROM products WHERE product_no = 1;

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

- See that product is deleted but order still exists

In [126]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
0 rows affected.


product_no,name,price


In [127]:
%sql SELECT * FROM orders;

 * postgresql://postgres@localhost/mydb
1 rows affected.


order_id,product_no,quantity
1,None,10


###### Deleting related records automatically with SET DEFAULT

- Using SET DEFAULT is risky.
- When setting default value of fk column, default value must reference any product otherwise error will be raised.

In [128]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer DEFAULT 1 REFERENCES products(product_no) ON DELETE SET DEFAULT,
    quantity integer
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting related records

In [129]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (2, 'product-2', 10);
INSERT INTO orders VALUES (1, 1, 10);
INSERT INTO orders VALUES (2, 2, 10);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [130]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
2 rows affected.


product_no,name,price
1,product-1,10
2,product-2,10


In [131]:
%sql SELECT * FROM orders;

 * postgresql://postgres@localhost/mydb
2 rows affected.


order_id,product_no,quantity
1,1,10
2,2,10


- Deleting product with id 2 will cause the change of order with product_no 2.
- New product_no value of order-2 will be 1. Since there is a product with product_no 1 there will be no error.

In [132]:
%sql DELETE FROM products WHERE product_no = 2;

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

In [133]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
1 rows affected.


product_no,name,price
1,product-1,10


In [134]:
%sql SELECT * FROM orders;

 * postgresql://postgres@localhost/mydb
2 rows affected.


order_id,product_no,quantity
1,1,10
2,1,10


- Now deleting product with id 1 will cause the change of orders with product_no 1.
- But since there is no record in products with product_no 1 an error will be raised.

In [135]:
%sql DELETE FROM products WHERE product_no = 1;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.ForeignKeyViolation) HATA:  "products" tablosu üzerinde yapılan update veya delete işlemi "orders" tablosunun "orders_product_no_fkey" bütünlük kısıtlamasını ihlal ediyor
DETAIL:  (product_no)=(1) anahtarı "orders" tablosundan hala referans edilmektedir.

[SQL: DELETE FROM products WHERE product_no = 1;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


###### Difference between NO ACTION and RESTRICTED

- NO ACTION can be described as DEFERRABLE but restricted can not.

- To see the effects in this section auto-commit will be disabled.
- Do not forget to re-open for other sections.

In [136]:
%config SqlMagic.autocommit=False

In [137]:
%%sql
BEGIN;
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer REFERENCES products(product_no) ON DELETE NO ACTION DEFERRABLE INITIALLY DEFERRED,
    quantity integer
);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.
Done.
Done.


[]

- Inserting related records

In [138]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO orders VALUES (1, 1, 10);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
1 rows affected.
1 rows affected.
Done.


[]

- In transaction first product then order can be deleted because FK check is deferred until transaction ends
- See that both product and order deleted

In [139]:
%%sql
BEGIN;
DELETE FROM products WHERE product_no = 1;
DELETE FROM orders WHERE product_no = 1;
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
1 rows affected.
1 rows affected.
Done.


[]

In [140]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
0 rows affected.


product_no,name,price


In [141]:
%sql SELECT * FROM orders;

 * postgresql://postgres@localhost/mydb
0 rows affected.


order_id,product_no,quantity


- Even when defining RESTRICT as DEFERRED, it has no effect on transaction.
- Still deleting product before delete order violates error even in transaction.
- So it behaves as RESTRICT even if you define it as DEFERRED.

In [142]:
%%sql
BEGIN;
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer REFERENCES products(product_no) ON DELETE RESTRICT DEFERRABLE INITIALLY DEFERRED,
    quantity integer
);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.
Done.
Done.


[]

- Inserting related records

In [143]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO orders VALUES (1, 1, 10);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
1 rows affected.
1 rows affected.
Done.


[]

- Delete product will raise error because FK constraint checks violation imeediately even if described as DEFERRED

In [144]:
%%sql
BEGIN;
DELETE FROM products WHERE product_no = 1;
DELETE FROM orders WHERE product_no = 1;
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
(psycopg2.errors.ForeignKeyViolation) HATA:  "products" tablosu üzerinde yapılan update veya delete işlemi "orders" tablosunun "orders_product_no_fkey" bütünlük kısıtlamasını ihlal ediyor
DETAIL:  (product_no)=(1) anahtarı "orders" tablosundan hala referans edilmektedir.

[SQL: DELETE FROM products WHERE product_no = 1;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [145]:
%config SqlMagic.autocommit=True

#### 5.5.6. Exclusion Constraints

- Exclusion constraint works like UNIQUE operator but it allows compare values to constraint other than equality operator.
- Normally, a UNIQUE constraint prevents duplicate values in a column.
- An Exclusion Constraint, on the other hand, enforces a "cannot exist at the same time" restriction using a specified operator.
- Usage is  <i>EXCLUDE USING gist (point(lat, lon) WITH ~=)</i>
    - When using exclude column will be indexed.
    - In here gist specifies Generalized Search Tree algorithm will be used in indexing.
    - Other alternatives: btree, spgist, brin.
    - But gist is more common.

In [146]:
%%sql
DROP TABLE IF EXISTS locations CASCADE;
CREATE TABLE locations (
    id SERIAL PRIMARY KEY,
    lat DOUBLE PRECISION NOT NULL,
    lon DOUBLE PRECISION NOT NULL,
    EXCLUDE USING gist (point(lat, lon) WITH ~=)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting same point will raise exception

In [147]:
%%sql
INSERT INTO locations VALUES (1, 10.0, 15.0);
INSERT INTO locations VALUES (2, 10.0, 15.0);

 * postgresql://postgres@localhost/mydb
1 rows affected.
(psycopg2.errors.ExclusionViolation) HATA:  uyuşmayan kayıt, "locations_point_excl" exclusion kısıtlamasını ihlal etmektedir
DETAIL:  (point(lat, lon))=((10,15)) anahtarı, mevcut (point(lat, lon))=((10,15)) anahtarıyla uyuşmuyor.

[SQL: INSERT INTO locations VALUES (2, 10.0, 15.0);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- Another example using date range

In [148]:
%%sql
CREATE EXTENSION IF NOT EXISTS btree_gist; 
DROP TABLE IF EXISTS reservations CASCADE;
CREATE TABLE reservations (
    id SERIAL PRIMARY KEY,
    room_id INTEGER NOT NULL,
    start_date DATE NOT NULL,
    end_date DATE NOT NULL,
    EXCLUDE USING gist (
        room_id WITH =,  -- Aynı oda numarasının tekrarını engeller
        daterange(start_date, end_date, '[]') WITH &&  -- Tarihler çakışamaz
    )
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting reservations

In [149]:
%%sql
INSERT INTO reservations (room_id, start_date, end_date) VALUES (1, '2025-03-01', '2025-03-04');
INSERT INTO reservations (room_id, start_date, end_date) VALUES (1, '2025-03-05', '2025-03-07');

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

- Inserting a record which dates collide with another record will raise error

In [150]:
%sql INSERT INTO reservations (room_id, start_date, end_date) VALUES (1, '2025-03-06', '2025-03-06');

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.ExclusionViolation) HATA:  uyuşmayan kayıt, "reservations_room_id_daterange_excl" exclusion kısıtlamasını ihlal etmektedir
DETAIL:  (room_id, daterange(start_date, end_date, '[]'::text))=(1, [2025-03-06,2025-03-07)) anahtarı, mevcut (room_id, daterange(start_date, end_date, '[]'::text))=(1, [2025-03-05,2025-03-08)) anahtarıyla uyuşmuyor.

[SQL: INSERT INTO reservations (room_id, start_date, end_date) VALUES (1, '2025-03-06' , '2025-03-06' );]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


### 5.7. Modifying Tables

#### 5.7.1. Adding a Column

- When adding new column you can specify default value with <i>DEFAULT</i>
- Updating column properties from documentation:
    - From PostgreSQL 11, adding a column with a constant default value no longer means that each row of the table needs to be updated when the ALTER TABLE statement is executed. Instead, the default value will be returned the next time the row is accessed, and applied when the table is rewritten, making the ALTER TABLE very fast even on large tables.
    - However, if the default value is volatile (e.g., clock_timestamp()) each row will need to be updated with the value calculated at the time ALTER TABLE is executed. To avoid a potentially lengthy update operation, particularly if you intend to fill the column with mostly nondefault values anyway, it may be preferable to add the column with no default, insert the correct values using UPDATE, and then add any desired default as described below.

In [151]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric,
    insert_time timestamp
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

In [152]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10, clock_timestamp());
INSERT INTO products VALUES (2, 'product-2', 10, clock_timestamp());
INSERT INTO products VALUES (3, 'product-3', 10, clock_timestamp());

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [153]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
3 rows affected.


product_no,name,price,insert_time
1,product-1,10,2025-03-10 21:55:05.468272
2,product-2,10,2025-03-10 21:55:05.470540
3,product-3,10,2025-03-10 21:55:05.471417


- According to documentation:
    - modified_time will be calculated and value applied immediately.
    - when selecting a product photo_url will be taken from DEFAULT value. The value is not applied to products until table is rewritten.
    - category values will be NULL because any default value did not specified.

In [154]:
%%sql 
ALTER TABLE products
    ADD COLUMN modified_time timestamp DEFAULT clock_timestamp(),
    ADD COLUMN photo_url text DEFAULT 'https://placehold.co/600x400/EEE/31343C',
    ADD COLUMN category text

 * postgresql://postgres@localhost/mydb
Done.


[]

In [155]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
3 rows affected.


product_no,name,price,insert_time,modified_time,photo_url,category
1,product-1,10,2025-03-10 21:55:05.468272,2025-03-10 21:55:05.504710,https://placehold.co/600x400/EEE/31343C,None
2,product-2,10,2025-03-10 21:55:05.470540,2025-03-10 21:55:05.504860,https://placehold.co/600x400/EEE/31343C,None
3,product-3,10,2025-03-10 21:55:05.471417,2025-03-10 21:55:05.504863,https://placehold.co/600x400/EEE/31343C,None


##### Constraints with adding column

- When giving constraint to new added column, you should ensure that DEFAULT value comply with constraint.

In [156]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

In [157]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (2, 'product-2', 10);
INSERT INTO products VALUES (3, 'product-3', 10);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
1 rows affected.


[]

- Giving NOT NULL constraing without specifying DEFAULT value raises error.

In [158]:
%sql ALTER TABLE products ADD COLUMN category text NOT NULL

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.NotNullViolation) HATA:  column "category" of relation "products" contains null values

[SQL: ALTER TABLE products ADD COLUMN category text NOT NULL]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


#### 5.7.2. Removing a Column

- Removing column will also remove the constraints defined on that column.
- If column is referenced by another table as foreign key drop operation will raise an error.
    - To prevent this use <i>CASCADE</i>, so FK constraint wil be dropped.

In [159]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

In [160]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (2, 'product-2', 10);
INSERT INTO products VALUES (3, 'product-3', 10);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [161]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
3 rows affected.


product_no,name,price
1,product-1,10
2,product-2,10
3,product-3,10


In [162]:
%sql ALTER TABLE products DROP COLUMN price;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [163]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
3 rows affected.


product_no,name
1,product-1
2,product-2
3,product-3


##### Remove column with constraints

In [164]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric CONSTRAINT constraint_price_check CHECK (price > 10)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- See that there is a constraint named <i>constraint_price_check</i>.

In [165]:
%%sql
SELECT 
    conname AS constraint_name,
    contype AS constraint_type
FROM 
    pg_catalog.pg_constraint
WHERE 
    conrelid = 'products'::regclass;

 * postgresql://postgres@localhost/mydb
2 rows affected.


constraint_name,constraint_type
constraint_price_check,c
products_pkey1,p


In [166]:
%sql ALTER TABLE products DROP COLUMN price;

 * postgresql://postgres@localhost/mydb
Done.


[]

- See that <i>constraint_price_check</i> is also dropped.

In [167]:
%%sql
SELECT 
    conname AS constraint_name,
    contype AS constraint_type
FROM 
    pg_catalog.pg_constraint
WHERE 
    conrelid = 'products'::regclass;

 * postgresql://postgres@localhost/mydb
1 rows affected.


constraint_name,constraint_type
products_pkey1,p


##### Remove column with table constraints which contains multiple column

In [168]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric,
    CONSTRAINT constraint_name_and_price CHECK (NAME <> '' AND price > 10)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- See that there is a single constraint named <i>constraint_name_and_price</i>.

In [169]:
%%sql
SELECT 
    conname AS constraint_name,
    contype AS constraint_type
FROM 
    pg_catalog.pg_constraint
WHERE 
    conrelid = 'products'::regclass;

 * postgresql://postgres@localhost/mydb
2 rows affected.


constraint_name,constraint_type
constraint_name_and_price,c
products_pkey1,p


In [170]:
%sql ALTER TABLE products DROP COLUMN price;

 * postgresql://postgres@localhost/mydb
Done.


[]

- See that <i>constraint_name_and_price</i> is also dropped.

In [171]:
%%sql
SELECT 
    conname AS constraint_name,
    contype AS constraint_type
FROM 
    pg_catalog.pg_constraint
WHERE 
    conrelid = 'products'::regclass;

 * postgresql://postgres@localhost/mydb
1 rows affected.


constraint_name,constraint_type
products_pkey1,p


##### Remove column with referenced by another table by FK constraint

In [172]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric,
    product_ref integer UNIQUE
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer REFERENCES products (product_ref),
    quantity integer
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- See that there is a foreign key constraint in orders table.

In [173]:
%%sql
SELECT 
    conname AS constraint_name,
    contype AS constraint_type,
    conrelid::regclass AS table_name
FROM 
    pg_catalog.pg_constraint
WHERE 
    conrelid IN ('products'::regclass, 'orders'::regclass);

 * postgresql://postgres@localhost/mydb
4 rows affected.


constraint_name,constraint_type,table_name
products_pkey1,p,products
products_product_ref_key,u,products
orders_pkey,p,orders
orders_product_no_fkey,f,orders


- When attempting to drop product_ref column which referenced from orders table by fk constraint, it raises error.

In [174]:
%sql ALTER TABLE products DROP COLUMN product_ref;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu'nin product_ref sütunu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu'nin product_ref sütunu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: ALTER TABLE products DROP COLUMN product_ref;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


- By using <i>CASCADE</i> product_ref column and fk constraint will be dropped.

In [175]:
%sql ALTER TABLE products DROP COLUMN product_ref CASCADE;

 * postgresql://postgres@localhost/mydb
Done.


[]

- See that foreign key constraint also dropped.

In [176]:
%%sql
SELECT 
    conname AS constraint_name,
    contype AS constraint_type,
    conrelid::regclass AS table_name
FROM 
    pg_catalog.pg_constraint
WHERE 
    conrelid IN ('products'::regclass, 'orders'::regclass);

 * postgresql://postgres@localhost/mydb
2 rows affected.


constraint_name,constraint_type,table_name
products_pkey1,p,products
orders_pkey,p,orders


#### 5.7.3. Adding a Constraint

- When adding check constraints to column, constraint will check all records immediately.
- To prevent this you can use NOT VALID
- Constraints Supporting NOT VALID in PostgreSQL
| Constraint Type   | Supports NOT VALID? | Description |
|------------------|----------------------|-------------|
| **CHECK**       | Yes | Allows adding a CHECK constraint without validating existing data. |
| **FOREIGN KEY** | Yes | Adds a FOREIGN KEY constraint without checking existing data. |
| **UNIQUE**      | No | NOT VALID is not supported for UNIQUE constraints. |
| **PRIMARY KEY** | No | NOT VALID is not supported for PRIMARY KEY constraints. |
| **EXCLUSION**   | No | NOT VALID is not applicable for EXCLUSION constraints. |
| **NOT NULL**    | No | NOT VALID cannot be used with ALTER COLUMN ... SET NOT NULL. |

In [177]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

In [178]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 5);
INSERT INTO products VALUES (2, 'product-2', 10);
INSERT INTO products VALUES (3, 'product-3', 15);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
1 rows affected.


[]

- When attempt to add constraint records checked immediately.
- Here error will be raised because some rows is violating new constraint.

In [179]:
%sql ALTER TABLE products ADD CONSTRAINT constraint_price_check CHECK (price > 10);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.CheckViolation) HATA:  check constraint "constraint_price_check" of relation "products" is violated by some row

[SQL: ALTER TABLE products ADD CONSTRAINT constraint_price_check CHECK (price > 10);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- When adding constraint with NOT VALID, old records would not be checked.
- But when adding new record, constraint will be applied.

In [180]:
%sql ALTER TABLE products ADD CONSTRAINT constraint_price_check CHECK (price > 10) NOT VALID;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [181]:
%sql INSERT INTO products VALUES (4, 'product-4', 5);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.CheckViolation) HATA:  "products" tablosuna girilen yeni satır "constraint_price_check" check kısıtlamasını ihlal ediyor
DETAIL:  Hata veren satır (4, product-4, 5) içeriyor.

[SQL: INSERT INTO products VALUES (4, 'product-4' , 5);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- You can manually check old records with using `VALIDATE CONSTRAINT`

In [182]:
%sql ALTER TABLE products VALIDATE CONSTRAINT constraint_price_check;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.CheckViolation) HATA:  check constraint "constraint_price_check" of relation "products" is violated by some row

[SQL: ALTER TABLE products VALIDATE CONSTRAINT constraint_price_check;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


#### 5.7.4. Removing a Constraint

In [183]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text NOT NULL,
    price numeric CONSTRAINT constraint_price_check CHECK (price > 10)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Drop constraint by using name.

In [184]:
%sql ALTER TABLE products DROP CONSTRAINT constraint_price_check;

 * postgresql://postgres@localhost/mydb
Done.


[]

- DROP column constraint.

In [185]:
%sql ALTER TABLE products ALTER name DROP NOT NULL;

 * postgresql://postgres@localhost/mydb
Done.


[]

#### 5.7.5. Changing a Column's Default Value

In [186]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Changing default value.

In [187]:
%sql ALTER TABLE products ALTER COLUMN name SET DEFAULT 'productX';

 * postgresql://postgres@localhost/mydb
Done.


[]

In [188]:
%%sql
SELECT column_name, column_default
FROM information_schema.columns
WHERE table_name = 'products';

 * postgresql://postgres@localhost/mydb
3 rows affected.


column_name,column_default
product_no,None
price,None
name,'productX'::text


- Removing default value.

In [189]:
%sql ALTER TABLE products ALTER COLUMN name DROP DEFAULT;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [190]:
%%sql
SELECT column_name, column_default
FROM information_schema.columns
WHERE table_name = 'products';

 * postgresql://postgres@localhost/mydb
3 rows affected.


column_name,column_default
product_no,None
price,None
name,None


#### 5.7.6. Changing a Column's Data Type

In [191]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

In [192]:
%sql ALTER TABLE products ALTER COLUMN price TYPE numeric(10,2);

 * postgresql://postgres@localhost/mydb
Done.


[]

#### 5.7.7. Renaming a Column

In [193]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

In [194]:
%sql ALTER TABLE products RENAME COLUMN product_no TO product_number;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [195]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
0 rows affected.


product_number,name,price


#### 5.7.8. Renaming a Table

In [196]:
%%sql
DROP TABLE IF EXISTS products CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

In [197]:
%sql ALTER TABLE products RENAME TO items;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DuplicateTable) HATA:  "items" nesnesi zaten mevcut

[SQL: ALTER TABLE products RENAME TO items;]
(Background on this error at: https://sqlalche.me/e/20/f405)


### 5.8. Privileges

- From documentation: `When an object is created, it is assigned an owner. The owner is normally the role that executed the
creation statement. For most kinds of objects, the initial state is that only the owner (or a superuser)
can do anything with the object. To allow other roles to use it, privileges must be granted.`

- First, create a test user to perform the tests.
- Here, `test_user` will be used as the username.
- If you created `test_user` before (or executed later queries and return here to execute again), you can either choose another username or drop `test_user`.
    - When removing the user, you will probably get an error because there are related privileges with `test_user`. So you should run this query first:
        ```sql
            REVOKE ALL PRIVILEGES ON ALL TABLES IN SCHEMA public FROM test_user;
            REVOKE ALL PRIVILEGES ON ALL SEQUENCES IN SCHEMA public FROM test_user;
            REVOKE ALL PRIVILEGES ON ALL FUNCTIONS IN SCHEMA public FROM test_user;
            REVOKE ALL PRIVILEGES ON SCHEMA public FROM test_user;
            ALTER DEFAULT PRIVILEGES IN SCHEMA public REVOKE ALL ON TABLES FROM test_user;
            ALTER DEFAULT PRIVILEGES IN SCHEMA public REVOKE ALL ON SEQUENCES FROM test_user;
            ALTER DEFAULT PRIVILEGES IN SCHEMA public REVOKE ALL ON FUNCTIONS FROM test_user;
            ALTER DEFAULT PRIVILEGES IN SCHEMA public REVOKE ALL ON TYPES FROM test_user;
        ```
    - You may still get an error because `test_user` owns some tables. You should transfer the ownership of the tables to another user or simply drop the tables.
    - To list the tables that `test_user` is the owner of:
        ```sql
            SELECT tablename 
            FROM pg_tables 
            WHERE tableowner = 'test_user' AND schemaname = 'public'; 
        ```
    - To transfer ownership:
        ```sql
            ALTER TABLE schema_name.table_name
            OWNER TO postgres; -- the username is up to you
        ```
- postgres user can make operations even if has no privilege granted. Because postgres is a superuser. You can give superuser privilege to other users.
    - Beside the operations all granting operations can be performed by superuser. 
    - To check if a user is superuser:
        ```sql
            SELECT usename, usesuper FROM pg_user WHERE usename = 'postgres';
        ```
- 

In [198]:
%%sql
SET ROLE postgres;
DROP ROLE IF EXISTS test_user, test_user_2, test_user_3;
CREATE ROLE test_user;
CREATE ROLE test_user_2;
CREATE ROLE test_user_3;

 * postgresql://postgres@localhost/mydb
Done.
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle "test_user" rolü kaldırılamıyor
DETAIL:  public şeması için yetkiler
privilege_sequence_example sequence nesnesinin sahibi
privilege_references_example tablosu nesnesinin sahibi
privilege_example tablosu nesnesinin sahibi

[SQL: DROP ROLE IF EXISTS test_user, test_user_2, test_user_3;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


- Granting privilege on `PUBLIC SCHEMA` to test_user to create tables.

In [199]:
%%sql 
GRANT ALL ON SCHEMA PUBLIC TO test_user;
GRANT ALL ON SCHEMA PUBLIC TO test_user_2;

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Switching to test_user user (Change to test_user role)
- From now on queries will be executed as test_user.

In [200]:
%sql SET ROLE test_user

 * postgresql://postgres@localhost/mydb
Done.


[]

- See that current user is test_user;

In [201]:
%sql SELECT current_user;

 * postgresql://postgres@localhost/mydb
1 rows affected.


current_user
test_user


#### Creating an object

- Creating an object generally refers to creating any kind of database object which includes `tables, views, indexes, schemas, functions, and other database entities`.
- Unless otherwise specified, examples related to creating objects will refer to tables, and the same properties will apply to other objects as well. Any exceptions to this will be explicitly mentioned.
- `When an object is created, it is assigned an owner`: Owner will be current user (user generally referred as role).
- Operation privileges for the table will be been assigned to test_user: `INSERT, SELECT, UPDATE, DELETE, TRUNCATE, REFERENCES and TRIGGER`

In [202]:
%sql SET ROLE test_user

 * postgresql://postgres@localhost/mydb
Done.


[]

In [203]:
%%sql
DROP TABLE IF EXISTS privilege_example;
CREATE TABLE privilege_example (
    id integer
);
INSERT INTO privilege_example VALUES (1);

 * postgresql://postgres@localhost/mydb
Done.
Done.
1 rows affected.


[]

- See that owner is test_user.

In [204]:
%sql SELECT tablename, tableowner FROM pg_tables where tablename = 'privilege_example';

 * postgresql://postgres@localhost/mydb
1 rows affected.


tablename,tableowner
privilege_example,test_user


- See that grantee for operation privileges is test_user.

In [205]:
%sql SELECT grantee, privilege_type FROM information_schema.role_table_grants WHERE table_name = 'privilege_example';

 * postgresql://postgres@localhost/mydb
7 rows affected.


grantee,privilege_type
test_user,INSERT
test_user,SELECT
test_user,UPDATE
test_user,DELETE
test_user,TRUNCATE
test_user,REFERENCES
test_user,TRIGGER


#### Granting privileges

##### Attempting to access with no privilege

- Before start granting privileges, first try to the access object with different role (user) which has no privilege on object.

In [206]:
%sql SET ROLE test_user;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [207]:
%%sql
DROP TABLE IF EXISTS privilege_example;
CREATE TABLE privilege_example (
    id integer
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

In [208]:
%sql SET ROLE test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

- See that access to privilege_example table is denied.
- Because test_user_2 role has no privilege on table.

In [209]:
%sql SELECT * FROM privilege_example

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InsufficientPrivilege) HATA:  privilege_example tablosuna erişim engellendi

[SQL: SELECT * FROM privilege_example]
(Background on this error at: https://sqlalche.me/e/20/f405)


##### Granting privileges

- Owner can perform granting operations.
- When owner grant privileges for another role, it does not mean grantee role can grant privileges for another roles.
- If the owner wants to grant privileges to other roles, they must use the `WITH GRANT OPTION` clause.
- There can be only single owner for database object.

##### Granting SELECT privileges

- Select privilege also allows to use of the `COPY TO` command for exporting data.
    - For more information check [https://www.postgresql.org/docs/17/sql-copy.html](https://www.postgresql.org/docs/17/sql-copy.html)

In [210]:
%sql SET ROLE test_user;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [211]:
%%sql
DROP TABLE IF EXISTS privilege_example;
CREATE TABLE privilege_example (
    id integer
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

In [212]:
%sql GRANT SELECT ON privilege_example TO test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [213]:
%sql SET ROLE test_user_2

 * postgresql://postgres@localhost/mydb
Done.


[]

In [214]:
%sql SELECT * FROM privilege_example;

 * postgresql://postgres@localhost/mydb
0 rows affected.


id


##### Granting SELECT privileges on SEQUENCE

- Select operation only allows `currval()` function of a sequence.
- USage of `nextval()` and `setval()` functions  will be denied if update privilege is not given, because these functions modifies the value of sequence.

In [215]:
%sql SET ROLE test_user;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [216]:
%%sql
DROP SEQUENCE IF EXISTS privilege_sequence_example;
CREATE SEQUENCE privilege_sequence_example;

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

In [217]:
%sql GRANT SELECT ON privilege_sequence_example TO test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [218]:
%sql SET ROLE test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [219]:
%sql SELECT currval('privilege_sequence_example')

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.ObjectNotInPrerequisiteState) HATA:  bu oturumda "privilege_sequence_example" sequence'i için currval henüz tanımlanmamıştır

[SQL: SELECT currval('privilege_sequence_example')]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


- See that nextval function is denied for test_user_2_role.

In [220]:
%sql SELECT nextval('privilege_sequence_example')

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InsufficientPrivilege) HATA:  privilege_sequence_example sequence'ine erişim izni verilmedi

[SQL: SELECT nextval('privilege_sequence_example')]
(Background on this error at: https://sqlalche.me/e/20/f405)


##### Granting INSERT privileges

- `INSERT` privilege can be granted for the entire table or for specific columns.
- If this privilege given for only selected columns, other columns will be assigned default values.
    - `Note:` If there are constraints it may raise error inserting default values.
- Insert privilege also allows use of `COPY FROM` command.
    - For more information check [https://www.postgresql.org/docs/17/sql-copy.html](https://www.postgresql.org/docs/17/sql-copy.html)

##### Granting INSERT privileges for table

In [221]:
%sql SET ROLE test_user;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [222]:
%%sql
DROP TABLE IF EXISTS privilege_example CASCADE;
CREATE TABLE privilege_example (
    id integer,
    username text
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

In [223]:
%sql GRANT INSERT ON privilege_example TO test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [224]:
%sql SET ROLE test_user_2

 * postgresql://postgres@localhost/mydb
Done.


[]

In [225]:
%sql INSERT INTO privilege_example VALUES (1, 'example-user-1');

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

##### Granting INSERT privileges for column

In [226]:
%sql SET ROLE test_user;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [227]:
%%sql
DROP TABLE IF EXISTS privilege_example CASCADE;
CREATE TABLE privilege_example (
    id integer,
    username text,
    role text DEFAULT ('new-user')
)

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Granted also for SELECT operation to see insertion results.

In [228]:
%%sql 
GRANT SELECT, INSERT (id, username) ON privilege_example TO test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [229]:
%sql SET ROLE test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [230]:
%sql INSERT INTO privilege_example(id, username) VALUES (1, 'example-user-1');

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

- See that role is assigned as the DEFAULT value.

In [231]:
%sql SELECT * FROM privilege_example;

 * postgresql://postgres@localhost/mydb
1 rows affected.


id,username,role
1,example-user-1,new-user


- See that specifying role raises error, because privilege did not granted.

In [232]:
%sql INSERT INTO privilege_example(id, username, role) VALUES (1, 'example-user-1', 'new-role-1');

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InsufficientPrivilege) HATA:  privilege_example tablosuna erişim engellendi

[SQL: INSERT INTO privilege_example(id, username, role) VALUES (1, 'example-user-1' , 'new-role-1' );]
(Background on this error at: https://sqlalche.me/e/20/f405)


##### Granting UPDATE privileges

- Granting UPDATE privilege allows update of table or columns.
- Mostly granting UPDATE requires granting SELECT since it determines which rows will be updated and/or compute new values for columns.
    - Even using `WHERE` clause causes error if `SELECT` operation is not granted.
- `SELECT ... FOR UPDATE` and `SELECT ... FOR SHARE` also requires this privilege.
    - For more information about this clauses (``) check [https://www.postgresql.org/docs/current/sql-select.html](https://www.postgresql.org/docs/current/sql-select.html#SQL-FOR-UPDATE-SHARE)

###### Granting UPDATE privileges for table

In [233]:
%sql SET ROLE test_user;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [234]:
%%sql
DROP TABLE IF EXISTS privilege_example;
CREATE TABLE privilege_example (
    id integer,
    username text,
    role text
);
INSERT INTO privilege_example VALUES (1, 'example-user-1', 'new-user');
INSERT INTO privilege_example VALUES (2, 'example-user-2', 'new-user');

 * postgresql://postgres@localhost/mydb
Done.
Done.
1 rows affected.
1 rows affected.


[]

In [235]:
%sql GRANT SELECT, UPDATE ON privilege_example TO test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

- See that for table and automatically for all columns UPDATE privilege has been granted. 

In [236]:
%%sql
SET ROLE postgres;
SELECT 
    grantee,
    table_name,
    privilege_type
FROM 
    information_schema.table_privileges
WHERE 
    table_name = 'privilege_example' and grantee = 'test_user_2' and privilege_type = 'UPDATE';

 * postgresql://postgres@localhost/mydb
Done.
1 rows affected.


grantee,table_name,privilege_type
test_user_2,privilege_example,UPDATE


In [237]:
%%sql
SELECT 
    grantee,
    table_name,
    column_name,
    privilege_type
FROM 
    information_schema.column_privileges
WHERE 
    table_name = 'privilege_example' and grantee = 'test_user_2' and privilege_type = 'UPDATE';

 * postgresql://postgres@localhost/mydb
3 rows affected.


grantee,table_name,column_name,privilege_type
test_user_2,privilege_example,id,UPDATE
test_user_2,privilege_example,role,UPDATE
test_user_2,privilege_example,username,UPDATE


In [238]:
%sql SET ROLE test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [239]:
%sql UPDATE privilege_example SET role = 'admin' WHERE id = 1;

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

- See that new role of user with id 1 is admin. 

In [240]:
%sql SELECT * FROM privilege_example;

 * postgresql://postgres@localhost/mydb
2 rows affected.


id,username,role
2,example-user-2,new-user
1,example-user-1,admin


###### Granting UPDATE privileges for table without SELECT privilege

In [241]:
%sql SET ROLE test_user;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [242]:
%%sql
DROP TABLE IF EXISTS privilege_example;
CREATE TABLE privilege_example (
    id integer,
    username text,
    role text
);
INSERT INTO privilege_example VALUES (1, 'example-user-1', 'new-user');
INSERT INTO privilege_example VALUES (2, 'example-user-2', 'new-user');

 * postgresql://postgres@localhost/mydb
Done.
Done.
1 rows affected.
1 rows affected.


[]

In [243]:
%sql GRANT UPDATE ON privilege_example TO test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [244]:
%sql SET ROLE test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

- Without where condition there will be no error.

In [245]:
%sql UPDATE privilege_example SET role = 'updated-role';

 * postgresql://postgres@localhost/mydb
2 rows affected.


[]

- With where condition it raises an error.

In [246]:
%sql UPDATE privilege_example SET role = 'admin' WHERE id = 1;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InsufficientPrivilege) HATA:  privilege_example tablosuna erişim engellendi

[SQL: UPDATE privilege_example SET role = 'admin' WHERE id = 1;]
(Background on this error at: https://sqlalche.me/e/20/f405)


###### Granting UPDATE privileges for column

In [247]:
%sql SET ROLE test_user;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [248]:
%%sql
DROP TABLE IF EXISTS privilege_example;
CREATE TABLE privilege_example (
    id integer,
    username text,
    role text
);
INSERT INTO privilege_example VALUES (1, 'example-user-1', 'new-user');
INSERT INTO privilege_example VALUES (2, 'example-user-2', 'new-user');

 * postgresql://postgres@localhost/mydb
Done.
Done.
1 rows affected.
1 rows affected.


[]

In [249]:
%sql GRANT UPDATE (role) ON privilege_example TO test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [250]:
%sql SET ROLE test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [251]:
%sql UPDATE privilege_example SET role = 'updated-role';

 * postgresql://postgres@localhost/mydb
2 rows affected.


[]

- See that update denied and raised error.

In [252]:
%sql UPDATE privilege_example SET username = 'default-user';

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InsufficientPrivilege) HATA:  privilege_example tablosuna erişim engellendi

[SQL: UPDATE privilege_example SET username = 'default-user' ;]
(Background on this error at: https://sqlalche.me/e/20/f405)


###### Granting UPDATE privileges for sequence

- With UPDATE privilege you can use `nextval()` and `setval()` functions.
- But without SELECT privilege you can not use `currval()`function.

In [253]:
%sql SET ROLE test_user;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [254]:
%%sql
DROP SEQUENCE IF EXISTS privilege_sequence_example;
CREATE SEQUENCE privilege_sequence_example;

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

In [255]:
%sql GRANT UPDATE ON privilege_sequence_example TO test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [256]:
%sql SET ROLE test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [257]:
%sql SELECT nextval('privilege_sequence_example');

 * postgresql://postgres@localhost/mydb
1 rows affected.


nextval
1


- See that currval function raises error because SELECT privilege is not granted.

In [258]:
%sql SELECT currval('privilege_sequence_example');

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InsufficientPrivilege) HATA:  privilege_sequence_example sequence'ine erişim izni verilmedi

[SQL: SELECT currval('privilege_sequence_example');]
(Background on this error at: https://sqlalche.me/e/20/f405)


##### Granting DELETE privileges

- Mostly granting DELETE requires granting SELECT since it determines which rows will be deleted.
    - Even using `WHERE` clause causes error if `SELECT` operation is not granted.

In [259]:
%sql SET ROLE test_user;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [260]:
%%sql
DROP TABLE IF EXISTS privilege_example;
CREATE TABLE privilege_example (
    id integer,
    username text,
    role text
);
INSERT INTO privilege_example VALUES (1, 'example-user-1', 'new-user');
INSERT INTO privilege_example VALUES (2, 'example-user-2', 'new-user');

 * postgresql://postgres@localhost/mydb
Done.
Done.
1 rows affected.
1 rows affected.


[]

In [261]:
%sql GRANT SELECT, DELETE ON privilege_example TO test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [262]:
%sql SET ROLE test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [263]:
%sql DELETE FROM privilege_example WHERE id=2;

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

##### Granting TRUNCATE privileges

In [264]:
%sql SET ROLE test_user;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [265]:
%%sql
DROP TABLE IF EXISTS privilege_example;
CREATE TABLE privilege_example (
    id integer,
    username text,
    role text
);
INSERT INTO privilege_example VALUES (1, 'example-user-1', 'new-user');
INSERT INTO privilege_example VALUES (2, 'example-user-2', 'new-user');

 * postgresql://postgres@localhost/mydb
Done.
Done.
1 rows affected.
1 rows affected.


[]

In [266]:
%sql GRANT TRUNCATE ON privilege_example TO test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [267]:
%sql SET ROLE test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [268]:
%sql TRUNCATE TABLE privilege_example;

 * postgresql://postgres@localhost/mydb
Done.


[]

##### Granting REFERENCES privileges

- Reference privilege can be granted for table or selected columns.

###### Granting REFERENCES for table

In [269]:
%%sql
SET ROLE test_user_2;
DROP TABLE IF EXISTS privilege_references_example;
SET ROLE test_user;
DROP TABLE IF EXISTS privilege_example;
CREATE TABLE privilege_example (
    id integer PRIMARY KEY,
    username text,
    role text
);

 * postgresql://postgres@localhost/mydb
Done.
(psycopg2.errors.InsufficientPrivilege) HATA:  privilege_references_example tablosunun sahibi olmalısınız

[SQL: DROP TABLE IF EXISTS privilege_references_example;]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [270]:
%sql GRANT REFERENCES ON privilege_example TO test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [271]:
%sql SET ROLE test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

In [272]:
%%sql
DROP TABLE IF EXISTS privilege_references_example;
CREATE TABLE privilege_references_example (
    id integer,
    privilege_example_id integer REFERENCES privilege_example(id)
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InsufficientPrivilege) HATA:  privilege_references_example tablosunun sahibi olmalısınız

[SQL: DROP TABLE IF EXISTS privilege_references_example;]
(Background on this error at: https://sqlalche.me/e/20/f405)


###### Attempting to give reference without REFERENCES privilege

In [273]:
%%sql
SET ROLE test_user_2;
DROP TABLE IF EXISTS privilege_references_example;
SET ROLE test_user;
DROP TABLE IF EXISTS privilege_example;
CREATE TABLE privilege_example (
    id integer PRIMARY KEY,
    username text,
    role text
);

 * postgresql://postgres@localhost/mydb
Done.
(psycopg2.errors.InsufficientPrivilege) HATA:  privilege_references_example tablosunun sahibi olmalısınız

[SQL: DROP TABLE IF EXISTS privilege_references_example;]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [274]:
%sql SET ROLE test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

- See that references raises error.

In [275]:
%%sql
DROP TABLE IF EXISTS privilege_references_example;
CREATE TABLE privilege_references_example (
    id integer,
    privilege_example_id integer REFERENCES privilege_example (id)
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InsufficientPrivilege) HATA:  privilege_references_example tablosunun sahibi olmalısınız

[SQL: DROP TABLE IF EXISTS privilege_references_example;]
(Background on this error at: https://sqlalche.me/e/20/f405)


###### Granting REFERENCES for column

In [276]:
%%sql
SET ROLE test_user_2;
DROP TABLE IF EXISTS privilege_references_example;
SET ROLE test_user;
DROP TABLE IF EXISTS privilege_example;
CREATE TABLE privilege_example (
    id integer PRIMARY KEY,
    username text UNIQUE,
    role text
);

 * postgresql://postgres@localhost/mydb
Done.
(psycopg2.errors.InsufficientPrivilege) HATA:  privilege_references_example tablosunun sahibi olmalısınız

[SQL: DROP TABLE IF EXISTS privilege_references_example;]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [277]:
%sql GRANT REFERENCES (id) ON privilege_example TO test_user_2;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InsufficientPrivilege) HATA:  "privilege_example" nesnesinin "id" sütununa erişim engellendi

[SQL: GRANT REFERENCES (id) ON privilege_example TO test_user_2;]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [278]:
%sql SET ROLE test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

- Reference constraint was added without error because references were granted for the id column.

In [279]:
%%sql
DROP TABLE IF EXISTS privilege_references_example;
CREATE TABLE privilege_references_example (
    id integer,
    privilege_example_id integer REFERENCES privilege_example(id)
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InsufficientPrivilege) HATA:  privilege_references_example tablosunun sahibi olmalısınız

[SQL: DROP TABLE IF EXISTS privilege_references_example;]
(Background on this error at: https://sqlalche.me/e/20/f405)


- Error raised when adding reference constraint because references were not granted for the username column.

In [280]:
%%sql
DROP TABLE IF EXISTS privilege_references_example;
CREATE TABLE privilege_references_example (
    id integer,
    privilege_example_username text REFERENCES privilege_example(username)
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InsufficientPrivilege) HATA:  privilege_references_example tablosunun sahibi olmalısınız

[SQL: DROP TABLE IF EXISTS privilege_references_example;]
(Background on this error at: https://sqlalche.me/e/20/f405)


##### Granting TRIGGER privileges

[TBA]

##### Granting CREATE privileges

- For databases, allows new schemas and publications to be created within the database, and allows trusted extensions to be installed within the database.
- For schemas, allows new objects to be created within the schema.
    - To rename an existing object, you must own the object and have this privilege for the containing schema.
- For tablespaces, allows tables, indexes, and temporary files to be created within the tablespace, and allows databases to be created that have the tablespace as their default tablespace.
- Note that revoking this privilege will not alter the existence or location of existing objects.

###### Granting CREATE privileges for DATABASE

[TBA]

###### Granting CREATE privileges for SCHEMA

[TBA]

###### Granting CREATE privileges for TABLESPACE

[TBA]

##### Granting TEMPORARY privileges

[TBA]

##### Granting EXECUTE privileges

[TBA]

##### Granting USAGE privileges

[TBA]

##### Granting SET privileges

[TBA]

##### Granting ALTER privileges

[TBA]

##### Granting MAINTAIN privileges

[TBA]

#### Revoking privileges

- Ownership can not be revoked. To change ownership you must use:
    ```sql
        ALTER TABLE privilege_example OWNER TO postgres;
    ```
- If privileges are revoked, dependent objects such as views, functions and foreign keys may break.
    - Check dependent views, functions and foreign keys before revoking.
- If a role was granted privileges using `WITH GRANT OPTION`, they might have granted these privileges to others.
    - In this case when you revoke privileges from a role, other roles which granted by that role also revoked cascaded.
    - So you should be careful when revoking privileges includes `WITH GRAND OPTION`. Inspect all roles which privileged for database object.
- If a privilege was granted to PUBLIC, revoking it from an individual user will not be effective.
    - To remove a privilege from PUBLIC:
    ```sql
        REVOKE SELECT ON table_name FROM PUBLIC;
    ```
- If a user has superuser privileges, using REVOKE will not be effective.
    - To remove superuser privileges:
    ```sql
        ALTER ROLE some_user NOSUPERUSER;
    ```
- You can revoke the privileges from owner table.
    - Since the owner has grant authority, they can re-grant privileges.
- Examples will focus on the SELECT privilege, but the same principles apply to other privileges. Special cases will be explicitly specified.

##### Changing owner

In [281]:
%%sql
SET ROLE test_user;
DROP TABLE IF EXISTS privilege_example;
CREATE TABLE privilege_example (
    id integer
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

In [282]:
%%sql 
SET ROLE postgres;
SELECT tablename, tableowner FROM pg_tables WHERE tablename = 'privilege_example' AND schemaname = 'public';

 * postgresql://postgres@localhost/mydb
Done.
1 rows affected.


tablename,tableowner
privilege_example,test_user


- See that owner changed to `test_user_2`.

In [283]:
%%sql 
SET ROLE postgres;
ALTER TABLE privilege_example OWNER TO test_user_2;
SELECT tablename, tableowner FROM pg_tables WHERE tablename = 'privilege_example' AND schemaname = 'public';

 * postgresql://postgres@localhost/mydb
Done.
Done.
1 rows affected.


tablename,tableowner
privilege_example,test_user_2


##### Revoking SELECT privilege

In [284]:
%%sql
SET ROLE postgres;
ALTER TABLE IF EXISTS privilege_example OWNER TO test_user;
SET ROLE test_user;
DROP TABLE IF EXISTS privilege_example;
CREATE TABLE privilege_example (
    id integer
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.
Done.
Done.


[]

In [285]:
%sql GRANT SELECT ON privilege_example TO test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

- See that SELECT privilege granted for `test_user_2`.

In [286]:
%sql SELECT grantee, privilege_type FROM information_schema.role_table_grants WHERE table_name = 'privilege_example' AND grantee = 'test_user_2';

 * postgresql://postgres@localhost/mydb
1 rows affected.


grantee,privilege_type
test_user_2,SELECT


In [287]:
%sql REVOKE SELECT ON privilege_example FROM test_user_2;

 * postgresql://postgres@localhost/mydb
Done.


[]

- See that privilege has been removed from `test_user_2`.

In [288]:
%sql SELECT grantee, privilege_type FROM information_schema.role_table_grants WHERE table_name = 'privilege_example' AND grantee = 'test_user_2';

 * postgresql://postgres@localhost/mydb
0 rows affected.


grantee,privilege_type


##### Revoking privilege with dependent view

- If you created any view from table, when you want to remove table you should first remove the view.
    - Otherwise you will get dependency error. 

In [289]:
%%sql
SET ROLE postgres;
ALTER TABLE IF EXISTS privilege_example OWNER TO test_user;
SET ROLE test_user;
DROP TABLE IF EXISTS privilege_example;
CREATE TABLE privilege_example (
    id integer
);
CREATE VIEW privilege_view_example AS 
    SELECT * FROM privilege_example;

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.
Done.
Done.
Done.


[]

- To find dependent views created from table `pg_views` used.

In [290]:
%sql SELECT viewname FROM pg_views WHERE definition LIKE '%privilege_example%';

 * postgresql://postgres@localhost/mydb
1 rows affected.


viewname
privilege_view_example


- See that attempting to drop table raises error.

In [291]:
%sql DROP TABLE IF EXISTS privilege_example;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle privilege_example tablosu kaldırılamıyor
DETAIL:  privilege_view_example view, privilege_example tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS privilege_example;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


- When removing view first then removing table will be successful.

In [292]:
%%sql
DROP VIEW IF EXISTS privilege_view_example;
DROP TABLE IF EXISTS privilege_example;

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

##### Revoking privilege with dependent function

- Revoking a table that is used in any function does not restrict the revoking operation.
    - You can revoke privileges or drop table without getting error.
    - But after revoke priveileges or remove table, calling function raises error.
- When removing a table or revoke privileges from a table to prevent errors when calling a function,
    - You should either manually manage the functions (remove or change the referenced table):
        - If you remove a table:
            - You may change function body to remove usage of table or use different table.
            - You may remove the function.
                - You may remove function manually before removing table.
                - You may remove table with CASCADE option.
        - If you revoke privileges from table:
            - You may revoke the execute privilege from function.
                - Note in postgres functions execution granted to public as default.
                - So you should revoke execute privilege from public.
                - Also if there are any grant privilege on function for interested role, revoke them.
            - You may remove the function.

In [293]:
%%sql
SET ROLE postgres;
ALTER TABLE IF EXISTS privilege_example OWNER TO test_user;
DROP FUNCTION IF EXISTS get_user_name;
SET ROLE test_user;
DROP TABLE IF EXISTS privilege_example;
CREATE TABLE privilege_example (
    id integer,
    username text
);
INSERT INTO privilege_example VALUES (1, 'first-user');
CREATE OR REPLACE FUNCTION get_user_name(user_id INT)
RETURNS TEXT AS $$
DECLARE
    user_name TEXT;
BEGIN
    SELECT username INTO user_name
    FROM privilege_example
    WHERE id = user_id;

    RETURN user_name;
END;
$$ LANGUAGE plpgsql;

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.
Done.
Done.
Done.
1 rows affected.
Done.


[]

- To find dependent function contains table `pg_proc` used.

In [294]:
%sql SELECT proname FROM pg_proc WHERE prosrc LIKE '%privilege_example%';

 * postgresql://postgres@localhost/mydb
1 rows affected.


proname
get_user_name


- Calling function successfully.

In [295]:
%sql SELECT get_user_name(1);

 * postgresql://postgres@localhost/mydb
1 rows affected.


get_user_name
first-user


In [296]:
%sql REVOKE SELECT ON privilege_example FROM test_user;

 * postgresql://postgres@localhost/mydb
Done.


[]

- Calling function raises error because SELECT `privilege_example` privilege has been revoked from `test_user`.

In [297]:
%sql SELECT get_user_name(1);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InsufficientPrivilege) HATA:  privilege_example tablosuna erişim engellendi
CONTEXT:  SQL deyimi: "SELECT username                    FROM privilege_example
    WHERE id = user_id"
get_user_name(integer) PL/pgSQL fonksiyonu, 5. satır, SQL ifadesi içinde

[SQL: SELECT get_user_name(1);]
(Background on this error at: https://sqlalche.me/e/20/f405)


- Listing all privileges on get_user_name function.

In [298]:
%sql SELECT grantee, privilege_type FROM information_schema.role_routine_grants WHERE routine_name = 'get_user_name';

 * postgresql://postgres@localhost/mydb
2 rows affected.


grantee,privilege_type
PUBLIC,EXECUTE
test_user,EXECUTE


- Revoking privileges on get_username from public and test_user.

In [299]:
%sql REVOKE EXECUTE ON FUNCTION get_user_name FROM test_user, PUBLIC;

 * postgresql://postgres@localhost/mydb
Done.


[]

- When attempt to access get_user_name function access function error will be raised

In [300]:
%sql SELECT get_user_name(1);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InsufficientPrivilege) HATA:  get_user_name fonksiyonuna erişim engellendi

[SQL: SELECT get_user_name(1);]
(Background on this error at: https://sqlalche.me/e/20/f405)


##### Revoking privilege with dependent foreign key

- When revoking privileges on a table, if there are any foreign keys dependent on the table, it will raise an error.
- To prevent this you may remove the foreign key constraints first.
- When you deleting table with dependent foreign key constrainst, it also will raise an error.
- To prevent this:
    - You may remove the foreign key constraints first.
    - Or you may remove the table with CASCADE option.

In [301]:
%%sql
SET ROLE postgres;
ALTER TABLE IF EXISTS privilege_example OWNER TO test_user;
ALTER TABLE IF EXISTS privilege_references_example OWNER TO test_user;
DROP FUNCTION IF EXISTS get_user_name;
SET ROLE test_user;
DROP TABLE IF EXISTS privilege_example, privilege_references_example CASCADE;
CREATE TABLE privilege_example (
    id integer PRIMARY KEY,
    username text
);
CREATE TABLE privilege_references_example (
    id integer,
    privilege_example_id integer REFERENCES privilege_example(id)
);
INSERT INTO privilege_example VALUES (1, 'first-user');

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
1 rows affected.


[]

In [302]:
%sql REVOKE SELECT ON privilege_example FROM test_user;

 * postgresql://postgres@localhost/mydb
Done.


[]

- When attempt to insert record which has foreign key value will raise error.
- Because select privilege on privilege_example revoked for test_user.

In [303]:
%sql INSERT INTO privilege_references_example VALUES (1, 1);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InsufficientPrivilege) HATA:  privilege_example tablosuna erişim engellendi
CONTEXT:  SQL deyimi: "SELECT 1 FROM ONLY "public"."privilege_example" x WHERE "id" OPERATOR(pg_catalog.=) $1 FOR KEY SHARE OF x"

[SQL: INSERT INTO privilege_references_example VALUES (1, 1);]
(Background on this error at: https://sqlalche.me/e/20/f405)


#### DEFAULT PRIVILEGES

- Removing default privileges do not automatically revoke privileges from existing tables. You must manually revoke privileges from database objects.
    - For example you give INSERT privilege for all tables to a role: ```sql ALTER DEFAULT PRIVILEGES IN SCHEMA public GRANT INSERT ON TABLES TO test_user; ```
    - After that you created some tables. Then revoked default privileges: ```sql ALTER DEFAULT PRIVILEGES IN SCHEMA public REVOKE INSERT ON TABLES TO test_user; ```
    - Existing tables still have INSERT ON privileges, so if you want to revoke them you must do it manually.

#### Privileges with WITH GRAND OPTION

- `WITH GRAND OPTION` gives user a privilege to grant another users.
    - When `test_user` granted privilege of `test_user_2` with `WITH GRANT OPTION`, `test_user_2` have privilege to grant another roles.
- Privilege chain case:
    - `test_user` granted privilege for `test_user_2` with `WITH GRANT OPTION`.
    - `test_user_2` granted privilege for `test_user_3`.
    - When revoking `WITH GRANT OPTION` privilege or completely given privilege from `test_user_2`, it also revoke privilege from `test_user_3`.

###### WITH GRANT OPTION

In [304]:
%%sql
SET ROLE postgres;
ALTER TABLE IF EXISTS privilege_example OWNER TO test_user;
DROP FUNCTION IF EXISTS get_user_name;
DROP TABLE IF EXISTS privilege_example CASCADE;
SET ROLE test_user;
CREATE TABLE privilege_example (
    id integer,
    username text
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [101]:
%%sql
SET ROLE test_user;
GRANT SELECT ON privilege_example TO test_user_2 WITH GRANT OPTION;
SET ROLE test_user_2;
GRANT SELECT ON privilege_example TO test_user_3;
SET ROLE postgres;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InsufficientPrivilege) HATA:  "test_user" rolü ayarlanması engellendi

[SQL: SET ROLE test_user;]
(Background on this error at: https://sqlalche.me/e/20/f405)


- `test_user_2` and `test_user_3` have select privileges.
- select privilege of `test_user_2` is also grantable. 

In [306]:
%%sql 
SELECT grantee, privilege_type, is_grantable 
    FROM information_schema.role_table_grants 
    WHERE table_name = 'privilege_example' AND grantee in ('test_user_2', 'test_user_3');

 * postgresql://postgres@localhost/mydb
2 rows affected.


grantee,privilege_type,is_grantable
test_user_2,SELECT,YES
test_user_3,SELECT,NO


- Privilege revoking from `test_user_2` also caused revoking `test_user_3` by using CASCADE.

In [307]:
%%sql 
SET ROLE test_user;
REVOKE SELECT ON privilege_example FROM test_user_2 CASCADE;
SET ROLE postgres;

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- See that privileges for both `test_user_2` and `test_user_3` removed.

In [308]:
%%sql 
SELECT grantee, privilege_type, is_grantable 
    FROM information_schema.role_table_grants 
    WHERE table_name = 'privilege_example' AND grantee in ('test_user_2', 'test_user_3');

 * postgresql://postgres@localhost/mydb
0 rows affected.


grantee,privilege_type,is_grantable


### 5.9. Row Security Policies

- Operations that apply to the whole table, such as TRUNCATE and REFERENCES, are not subject to row security.
- BYPASSRLS
    - A user/role can bypass the row level security by using BYPASSRLS authority.
    - This means even if you give row level security rules to tables, that role can bypass this and run query without restriction.
    - For example if you want to give BYPASSRLS authority to all user which has admin role:
        - ```sql
          ALTER ROLE admin WITH BYPASSRLS;
          ```
    - superusers do not need to BYPASSRLS authority.
        - If a user is superuser, user can bypass RLS even is BYPASSRLS is not active
    - To see if a user has BYPASSRLS authority:
        - ```sql
          SELECT rolname, rolbypassrls FROM pg_roles WHERE rolname = 'test_user';
          ```
- For table owners
    - Table owners normally bypass row security as well, though a table owner can choose to be subject to row security with
        - ```sql
          ALTER TABLE ... FORCE ROW LEVEL SECURITY.
          ```
- Enabling and disabling row security
    - Enabling and disabling row security, as well as adding policies to a table, is always the privilege of the table owner only.
    - Table owner can not give the enabling and disabling row security authority to another user.
    - There is an exception here, superusers can enable and disable row security even if they are not the table owner.
- Policy roles
    - If no role is specified in the CREATE POLICY, it applies to all users by default.
        - ```sql
          CREATE POLICY user_policy ON users USING (user_name = current_user);
          ```
    - If you want the specify all users explicitly you can use `TO PUBLIC`, it will be more readable.
        - ```sql
          CREATE POLICY user_policy ON users TO PUBLIC USING (user_name = current_user);
          ```
    - Those two usage are equivalent.
    - You can check the roles by:
        - ```sql
          SELECT tablename, policyname, roles FROM pg_policies WHERE policyname = 'user_policy';
          ```
        - In both scenario roles are public.
- Permissive and restrictive policies
    - Permissive policies combined by or operator.
    - Restrictive policies combined by and operator.
    - If there are both permissive and restrictive policies
        - policy1 permissive, policy2 permissive, policy3 restrictive, policy4 restrictive
        - `(policy1 OR policy2) AND (policy3 AND policy4)`
    - By default all policies are permissive.

#### Roles Preperation

In [102]:
%%sql
SET SESSION AUTHORIZATION 'postgres';
DROP ROLE IF EXISTS rls_test_admin, rls_test_user1, rls_test_user2, user_role, admin_role;
CREATE ROLE user_role;
CREATE ROLE admin_role;
CREATE ROLE rls_test_user1;
CREATE ROLE rls_test_user2;
CREATE ROLE rls_test_admin;
GRANT user_role TO rls_test_user1, rls_test_user2;
GRANT admin_role TO rls_test_admin;

 * postgresql://postgres@localhost/mydb
Done.
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle "user_role" rolü kaldırılamıyor
DETAIL:  rls_example tablosu için yetkiler
rls_example tablosu üzerinde user_policy politikası 'nin hedefi

[SQL: DROP ROLE IF EXISTS rls_test_admin, rls_test_user1, rls_test_user2, user_role, admin_role;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


#### Row Security Policies Usage

- Creating table rls_example and grant select privilege to users and admins
- Created policies:
    - user_role can read only if username is current user.
    - admin role can read whole table.

In [108]:
%%sql
SET SESSION AUTHORIZATION 'postgres';
DROP TABLE IF EXISTS rls_example CASCADE;
CREATE TABLE rls_example (
    id integer PRIMARY KEY,
    username text
);
INSERT INTO rls_example VALUES (1, 'admin');
INSERT INTO rls_example VALUES (2, 'rls_test_user1');
INSERT INTO rls_example VALUES (3, 'rls_test_user2');
GRANT SELECT ON rls_example TO user_role, admin_role;

ALTER TABLE rls_example ENABLE ROW LEVEL SECURITY;
CREATE POLICY user_policy ON rls_example TO user_role USING (username = current_user); 
CREATE POLICY admin_policy ON rls_example TO admin_role USING (TRUE);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
Done.
Done.


[]

- Testing with a user with user_role.
- rls_test_user1 is an user and user can see only record with same username.

In [109]:
%%sql 
SET SESSION AUTHORIZATION rls_test_user1;
SELECT * FROM rls_example;

 * postgresql://postgres@localhost/mydb
Done.
1 rows affected.


id,username
2,rls_test_user1


- Testing with a user with admin_role.
- rls_test_admin is an admin and admin can see all records.

In [110]:
%%sql 
SET SESSION AUTHORIZATION rls_test_admin;
SELECT * FROM rls_example;

 * postgresql://postgres@localhost/mydb
Done.
3 rows affected.


id,username
1,admin
2,rls_test_user1
3,rls_test_user2


- Disabling RLS

In [113]:
%%sql 
SET SESSION AUTHORIZATION postgres;
ALTER TABLE rls_example DISABLE ROW LEVEL SECURITY;

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- After disabling rls, all users can read all records.

In [114]:
%%sql 
SET SESSION AUTHORIZATION rls_test_user1;
SELECT * FROM rls_example;

 * postgresql://postgres@localhost/mydb
Done.
3 rows affected.


id,username
1,admin
2,rls_test_user1
3,rls_test_user2


#### BYPASSRLS Usage

- Creating table rls_example and grant select privilege to users and admins
- Created policies:
    - user_role can read only if username is current user.
- Giving admin_role BYPASSRLS authority instead of creating permissive policy.

In [104]:
%%sql
SET SESSION AUTHORIZATION 'postgres';
DROP TABLE IF EXISTS rls_example CASCADE;
CREATE TABLE rls_example (
    id integer PRIMARY KEY,
    username text
);
INSERT INTO rls_example VALUES (1, 'admin');
INSERT INTO rls_example VALUES (2, 'rls_test_user1');
INSERT INTO rls_example VALUES (3, 'rls_test_user2');
GRANT SELECT ON rls_example TO user_role, admin_role;

ALTER TABLE rls_example ENABLE ROW LEVEL SECURITY;
CREATE POLICY user_policy ON rls_example TO user_role USING (username = current_user); 
ALTER ROLE admin_role WITH BYPASSRLS;

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
Done.
Done.


[]

- See admin_role has BYPASSRLS authority

In [107]:
%sql SELECT rolname, rolbypassrls FROM pg_roles WHERE rolname = 'admin_role';

 * postgresql://postgres@localhost/mydb
1 rows affected.


rolname,rolbypassrls
admin_role,True


- Testing with a user with admin_role.
- rls_test_admin has admin_role and admin_role has bypass authority on RLS.

In [106]:
%%sql 
SET SESSION AUTHORIZATION rls_test_admin;
SELECT * FROM rls_example;

 * postgresql://postgres@localhost/mydb
Done.
3 rows affected.


id,username
1,admin
2,rls_test_user1
3,rls_test_user2


- Testing with a superuser.
- postgres is a superuser and superuser can bypass row level security and see all records.

In [97]:
%%sql 
SET SESSION AUTHORIZATION postgres;
SELECT * FROM rls_example;

 * postgresql://postgres@localhost/mydb
Done.
3 rows affected.


id,username
1,admin
2,rls_test_user1
3,rls_test_user2


#### Permissive and Restrictive RLS

In [115]:
%%sql
SET SESSION AUTHORIZATION 'postgres';
DROP TABLE IF EXISTS rls_example_employee CASCADE;
CREATE TABLE rls_example_employee (
    employee_id SERIAL PRIMARY KEY,
    name TEXT,
    department TEXT,
    is_active BOOLEAN,
    has_signed_nondisclosure BOOLEAN
);
INSERT INTO rls_example_employee VALUES (1, 'employee-1', 'HR', TRUE, TRUE);
INSERT INTO rls_example_employee VALUES (2, 'employee-2', 'HR', TRUE, FALSE);
INSERT INTO rls_example_employee VALUES (3, 'employee-3', 'HR', FALSE, TRUE);
INSERT INTO rls_example_employee VALUES (4, 'employee-4', 'HR', FALSE, FALSE);
INSERT INTO rls_example_employee VALUES (5, 'employee-5', 'FINANCE', TRUE, TRUE);
INSERT INTO rls_example_employee VALUES (6, 'employee-6', 'FINANCE', TRUE, FALSE);
INSERT INTO rls_example_employee VALUES (7, 'employee-7', 'FINANCE', FALSE, TRUE);
INSERT INTO rls_example_employee VALUES (8, 'employee-8', 'FINANCE', FALSE, FALSE);
INSERT INTO rls_example_employee VALUES (9, 'employee-9', 'ENGINEERING', TRUE, TRUE);
INSERT INTO rls_example_employee VALUES (10, 'employee-10', 'ENGINEERING', TRUE, FALSE);
INSERT INTO rls_example_employee VALUES (11, 'employee-11', 'ENGINEERING', FALSE, TRUE);
INSERT INTO rls_example_employee VALUES (12, 'employee-12', 'ENGINEERING', FALSE, FALSE);
GRANT SELECT ON rls_example_employee TO user_role, admin_role;

ALTER TABLE rls_example_employee ENABLE ROW LEVEL SECURITY;
CREATE POLICY hr_department_policy ON rls_example_employee TO PUBLIC USING (department = 'HR'); 
CREATE POLICY finance_department_policy ON rls_example_employee TO PUBLIC USING (department = 'FINANCE'); 
CREATE POLICY active_employee_policy ON rls_example_employee AS RESTRICTIVE TO PUBLIC USING (is_active = TRUE); 
CREATE POLICY signed_nondisclosure_policy ON rls_example_employee AS RESTRICTIVE TO PUBLIC USING (has_signed_nondisclosure = TRUE); 

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
Done.
Done.
Done.
Done.


[]

- Evaluation policies is like:
    - `(hr_department_policy OR finance_department_policy) AND (active_employee_policy AND signed_nondisclosure_policy)`.
    - So employees only active and signed nondisclosure policy employees which are in HR or FINANCE department will be selected.

In [116]:
%%sql 
SET SESSION AUTHORIZATION rls_test_user1;
SELECT * FROM rls_example_employee;

 * postgresql://postgres@localhost/mydb
Done.
2 rows affected.


employee_id,name,department,is_active,has_signed_nondisclosure
1,employee-1,HR,True,True
5,employee-5,FINANCE,True,True


### 5.10. Schemas

[TBA]